# Using Historical Data to Predict Batting Success

Authored by: Donna J. Harris (994042890)

Email: harr2890@mylaurier.ca

For: CP640 Machine Learning (S22) with Professor Elham Harirpoush

## Introductory Comments

Based on the project proposal of the same name, this Jupyter Notebook demonstrates the process and exploration of Major League Baseball batting data from 1901 to 2021 to the end of discovering how or if historical data can be used to predict batting success.

The Kaggle dataset being used as the primary data source can be found in the `data` folder of the project folder structure: `./data/mlbbatting1901-2021.csv`

Each data record in the original dataset represents an individual batter's performance in a single game. This is why there are so many records. In a single game, there will be at least 18 batters with plate appearances across both teams, and often more with player substitutions, especially across extra innings.

## Environment Setup

Import and establish environment for initial work, including showing all dataframe column values.

In [1]:
import pandas as pd

pd.set_option('display.max_columns', None)

## Preprocessing

### Original Data

Acquire the data from the Kaggle dataset and place into a dataframe.

In [2]:
original_data_source = "./data/mlbbatting1901-2021.csv"

df = pd.read_csv(original_data_source)

Confirm the original data has been loaded into the data frame. (It should start with the earliest records from 1901 and end with the latest records from 2021.)

In [3]:
print(df)

                ID            Player        Date   Tm  Opp    Rslt  PA  AB  R  \
0        crossmo01       Monte Cross  1901-04-18  PHI  BRO  L 7-12   5   4  2   
1        dahlebi01       Bill Dahlen  1901-04-18  BRO  PHI  W 12-7   5   4  2   
2         dalyto01          Tom Daly  1901-04-18  BRO  PHI  W 12-7   5   5  1   
3        davisle01       Lefty Davis  1901-04-18  BRO  PHI  W 12-7   5   5  1   
4        delahed01      Ed Delahanty  1901-04-18  PHI  BRO  L 7-12   5   4  1   
...            ...               ...         ...  ...  ...     ...  ..  .. ..   
4285624  woodfja01     Jake Woodford  2021-10-03  STL  CHC   L 2-3   2   1  0   
4285625  yastrmi01  Mike Yastrzemski  2021-10-03  SFG  SDP  W 11-4   4   3  1   
4285626  zimmebr01    Bradley Zimmer  2021-10-03  CLE  TEX   W 6-0   4   4  1   
4285627  zimmery01    Ryan Zimmerman  2021-10-03  WSN  BOS   L 5-7   4   3  0   
4285628  zuninmi01       Mike Zunino  2021-10-03  TBR  NYY   L 0-1   4   4  0   

         H  2B  3B  HR  RBI

In [4]:
print(df.columns)

Index(['ID', 'Player', 'Date', 'Tm', 'Opp', 'Rslt', 'PA', 'AB', 'R', 'H', '2B',
       '3B', 'HR', 'RBI', 'BB', 'IBB', 'SO', 'HBP', 'SH', 'SF', 'ROE', 'GDP',
       'SB', 'CS', 'WPA', 'RE24', 'aLI', 'BOP', 'Pos Summary', 'DFS(DK)',
       'DFS(FD)'],
      dtype='object')


We have our confirmation that there are 31 feature columns and 4,285,629 data records.

### Extract Data

There is some data that might be of use to us that is trapped in existing columns. First, we want to extract the result of the game and the score of the game from the `'Rslt'` column. Then, we want to extract the year the game was played (denoting the season) from the `'Date'` column.

In [5]:
df[['Result','Score']] = df['Rslt'].str.split(' ', expand=True)

df['Season'] = df['Date'].str[:4]

### Column Removal

There are a number of columns in the original dataset that we know before going any further that we have no use for.

We no longer need `'Rslt'`, as we just split its interesting information into separate columns. Likewise with `'Date'`, we have what we need in the new `'Season'` column, so we can remove the `'Date'` column.

In [6]:
del df['Rslt']

In [7]:
del df['Date']

Daily fantasy sports points (used for fantasy leagues and betting) have no purpose within this project, so we can safely remove `'DFS(DK)'` and `'DFS(FD)'` from the data.

In [8]:
del df['DFS(DK)']
del df['DFS(FD)']

Similarly, to reduce complexity, we are not considering any statistics relating to fielding or base running/stealing. As such, we can remove `'SB'` and `'CS'`, which are the number of stolen bases and time caught stealing, respectively, as well as the `'Pos Summary'` (position summary) data.

In [9]:
del df['SB']
del df['CS']
del df['Pos Summary']

To further reduce complexity, we will remove the `'IBB'` (intentional walks) column as this is a subset of values tracked under the walks column (`'BB'`).  (Reference: https://en.wikipedia.org/wiki/Base_on_balls#Intentional_base_on_balls)

In [10]:
del df['IBB']

We can remove the `'GDP'` column, which represents the number of times a player hits into a double play (two outs). While this statistic has some bearing to the success of a batter, for this project we will exclude this nuance and focus more on the aspects of run production and getting on base through other statistical means.

In [11]:
del df['GDP']

Similarly to `'GDP'`, we will disregard the `'ROE'` column, which represents the number of times a player reaches base due to a fielding error by the opposing team. Like `'GDP'`, this statistic has some bearing to the success of a batter but it leans more toward their running abilities and street smarts of the player, as well as a bit of luck. Again, for this project, these nuances will be excluded for simplicity.

In [12]:
del df['ROE']

Before continuing on to value checking, let's look and see where the data is at after these processing operations.

In [13]:
print(df)

                ID            Player   Tm  Opp  PA  AB  R  H  2B  3B  HR  RBI  \
0        crossmo01       Monte Cross  PHI  BRO   5   4  2  2   1   0   0  1.0   
1        dahlebi01       Bill Dahlen  BRO  PHI   5   4  2  3   0   0   0  0.0   
2         dalyto01          Tom Daly  BRO  PHI   5   5  1  2   1   0   0  3.0   
3        davisle01       Lefty Davis  BRO  PHI   5   5  1  1   0   0   0  0.0   
4        delahed01      Ed Delahanty  PHI  BRO   5   4  1  2   0   0   0  0.0   
...            ...               ...  ...  ...  ..  .. .. ..  ..  ..  ..  ...   
4285624  woodfja01     Jake Woodford  STL  CHC   2   1  0  0   0   0   0  0.0   
4285625  yastrmi01  Mike Yastrzemski  SFG  SDP   4   3  1  1   1   0   0  2.0   
4285626  zimmebr01    Bradley Zimmer  CLE  TEX   4   4  1  2   0   0   0  1.0   
4285627  zimmery01    Ryan Zimmerman  WSN  BOS   4   3  0  0   0   0   0  1.0   
4285628  zuninmi01       Mike Zunino  TBR  NYY   4   4  0  0   0   0   0  0.0   

         BB  SO  HBP  SH   

In [14]:
print(df.columns)

Index(['ID', 'Player', 'Tm', 'Opp', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR',
       'RBI', 'BB', 'SO', 'HBP', 'SH', 'SF', 'WPA', 'RE24', 'aLI', 'BOP',
       'Result', 'Score', 'Season'],
      dtype='object')


We added three new columns (`'Result'`, `'Score'`, and `'Season'`) -- which we may or may not need later -- and also removed ten existing columns. So, this looks correct: 31 original features + 3 new features - 11 features = 23 feature columns.

We still have 4,285,629 data records, as we have not removed any records yet.

### Value Checking and Data Validation

**ID - Player ID**

Should be unique to each player. This will indicate how many players we currently have, based on player ID.

In [15]:
print(pd.unique(df['ID']))

print("\nNumber of unique player IDs: ", len(pd.unique(df['ID'])))

['crossmo01' 'dahlebi01' 'dalyto01' ... 'adonjo01' 'paynety01' 'stridsp01']

Number of unique player IDs:  15985


**Player - Player Name**

Should be roughly the same number of player IDs. Discrepancies are possible, errors in spelling, etc. but this is not highly significant for our statistics since we will key everything on the more reliable Player ID. We will hang onto this field to help humanly identify players by name. 

In [16]:
print(pd.unique(df['Player']))

print("\nNumber of unique player names: ", len(pd.unique(df['Player'])))

['Monte Cross' 'Bill Dahlen' 'Tom Daly' ... 'Joan Adon' 'Tyler Payne'
 'Spencer Strider']

Number of unique player names:  15595


**Note:** We found there were 15,985 unique IDs and 15,595 unique player names, which is a difference of 390 in favour of the IDs. This could easily be explained by different players over the years with the same name. This is a nuance that will be disregarded for this project. As previously stated, we will use the ID for all but human identification purposes.

**Season - Year the game was played in**
(Extracted from `'Date'`)

The values should all be visibly in YYYY format between the years of 1901 and 2021, inclusive.

Ideally, these values will be Integers -- but they have started as strings.

In [17]:
print(pd.unique(df['Season']))

['1901' '1902' '1903' '1904' '1905' '1906' '1907' '1908' '1909' '1910'
 '1911' '1912' '1913' '1914' '1915' '1916' '1917' '1918' '1919' '1920'
 '1921' '1922' '1923' '1924' '1925' '1926' '1927' '1928' '1929' '1930'
 '1931' '1932' '1933' '1934' '1935' '1936' '1937' '1938' '1939' '1940'
 '1941' '1942' '1943' '1944' '1945' '1946' '1947' '1948' '1949' '1950'
 '1951' '1952' '1953' '1954' '1955' '1956' '1957' '1958' '1959' '1960'
 '1961' '1962' '1963' '1964' '1965' '1966' '1967' '1968' '1969' '1970'
 '1971' '1972' '1973' '1974' '1975' '1976' '1977' '1978' '1979' '1980'
 '1981' '1982' '1983' '1984' '1985' '1986' '1987' '1988' '1989' '1990'
 '1991' '1992' '1993' '1994' '1995' '1996' '1997' '1998' '1999' '2000'
 '2001' '2002' '2003' '2004' '2005' '2006' '2007' '2008' '2009' '2010'
 '2011' '2012' '2013' '2014' '2015' '2016' '2017' '2018' '2019' '2020'
 '2021']


We can see visibly that these values are all YYYY integers, so let's convert them to actual integers in the dataframe.

In [18]:
df['Season'] = df['Season'].astype(int)
# test
print(pd.unique(df['Season']))

[1901 1902 1903 1904 1905 1906 1907 1908 1909 1910 1911 1912 1913 1914
 1915 1916 1917 1918 1919 1920 1921 1922 1923 1924 1925 1926 1927 1928
 1929 1930 1931 1932 1933 1934 1935 1936 1937 1938 1939 1940 1941 1942
 1943 1944 1945 1946 1947 1948 1949 1950 1951 1952 1953 1954 1955 1956
 1957 1958 1959 1960 1961 1962 1963 1964 1965 1966 1967 1968 1969 1970
 1971 1972 1973 1974 1975 1976 1977 1978 1979 1980 1981 1982 1983 1984
 1985 1986 1987 1988 1989 1990 1991 1992 1993 1994 1995 1996 1997 1998
 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012
 2013 2014 2015 2016 2017 2018 2019 2020 2021]


**Tm - Player's Team** and **Opp - Opponent**

The team values (both the player's team and the opposing team) should all be visibly in ZZZ format, belonging to a recognizable team between the 1901-2021 seasons. These are fields that will likely get dropped later on, but I'm keeping them until I know for sure I don't want them.

In [19]:
print("Player's Team (Tm):\n", pd.unique(df['Tm']))
print("\nOpposing Team (Opp):\n", pd.unique(df['Opp']))

Player's Team (Tm):
 ['PHI' 'BRO' 'BSN' 'NYG' 'STL' 'CHC' 'PIT' 'CIN' 'CLE' 'CHW' 'MLA' 'DET'
 'PHA' 'WSH' 'BOS' 'BLA' 'SLB' 'NYY' 'BUF' 'BAL' 'PBS' 'BTT' 'CHI' 'IND'
 'SLM' 'KCP' 'NEW' 'MLN' 'KCA' 'LAD' 'SFG' 'WSA' 'MIN' 'LAA' 'HOU' 'NYM'
 'CAL' 'ATL' 'OAK' 'KCR' 'MON' 'SDP' 'SEP' 'MIL' 'TEX' 'SEA' 'TOR' 'FLA'
 'COL' 'ANA' 'ARI' 'TBD' 'WSN' 'TBR' 'MIA']

Opposing Team (Opp):
 ['BRO' 'PHI' 'NYG' 'BSN' 'CHC' 'STL' 'CIN' 'PIT' 'CHW' 'CLE' 'DET' 'MLA'
 'WSH' 'PHA' 'BLA' 'BOS' 'SLB' 'NYY' 'BAL' 'BUF' 'BTT' 'PBS' 'KCP' 'SLM'
 'IND' 'CHI' 'NEW' 'MLN' 'KCA' 'SFG' 'LAD' 'WSA' 'MIN' 'LAA' 'HOU' 'NYM'
 'CAL' 'ATL' 'OAK' 'MON' 'SDP' 'KCR' 'SEP' 'MIL' 'TEX' 'SEA' 'TOR' 'COL'
 'FLA' 'ANA' 'ARI' 'TBD' 'WSN' 'TBR' 'MIA']


**PA - Plate Appearances**

Appearances should be an integer value, between the range of 1 and some upper value. (A 0 would indicate the player didn't bat in the game which would mean there should not be a record.)

The upper value will vary, although (speaking as a baseball fan) five plate appearances is pretty standard in a regular, nine-inning, low scoring game. But as soon as you get into higher scores and/or extra inning games, players can be up to bat many times.

In [20]:
print(pd.unique(df['PA']))

[ 5  4  3  1  2  6  7  8  9 10 11 12]


**Note:** 12 was the upper value. I'm curious of what era these are from, so let's take a look:

In [21]:
print(df[df['PA'] == 12])

                ID         Player   Tm  Opp  PA  AB  R  H  2B  3B  HR  RBI  \
2062446  millafe01   Felix Millan  NYM  STL  12  10  1  4   0   0   0  0.0   
2062447  milnejo01    John Milner  NYM  STL  12  10  0  2   1   0   0  1.0   
2458599  baineha01  Harold Baines  CHW  MIL  12  10  1  2   1   0   1  1.0   
2458643   fiskca01   Carlton Fisk  CHW  MIL  12  11  1  3   1   0   0  1.0   
2458686    lawru01       Rudy Law  CHW  MIL  12  11  1  4   0   0   0  1.0   

         BB  SO  HBP  SH   SF    WPA   RE24    aLI  BOP Result Score  Season  
2062446   1   0    0   1  0.0  0.060  0.422  1.894    2      L   3-4    1974  
2062447   2   3    0   0  0.0 -0.250 -0.399  2.147    4      L   3-4    1974  
2458599   2   0    0   0  0.0  0.195 -0.083  2.204    5      W   7-6    1984  
2458643   1   3    0   0  0.0 -0.237 -0.649  2.455    2      W   7-6    1984  
2458686   1   0    0   0  0.0  0.511  1.816  1.958    1      W   7-6    1984  


**AB - At Bats**

Similarly to Plate Appearances, At Bats should be an integer value. It should be between the range of 0 and some upper value. (Here, a 0 would indicate the player had one plate appearance that did not statistically count as an At Bat, such as a walk.)

The upper range should follow, and not exceed the upper value of Plate Appearances, which was 12. Note that 12 is possible, but does not have to be a value in this collection of data.

In [22]:
print(pd.unique(df['AB']))

[ 4  5  3  2  1  6  0  7  8  9 10 11]


**Note:** 11 was the upper value, which is less then 12. (The max number of plate appearances.)

We should validate that there are no records where there are more At Bats than Plate Appearances.

In [23]:
print("-----------------------------------------")
print("All records with PA < AB")
print("-----------------------------------------")
print(df[df['PA'] < df['AB']])

-----------------------------------------
All records with PA < AB
-----------------------------------------
Empty DataFrame
Columns: [ID, Player, Tm, Opp, PA, AB, R, H, 2B, 3B, HR, RBI, BB, SO, HBP, SH, SF, WPA, RE24, aLI, BOP, Result, Score, Season]
Index: []


**Note:** The validation checks pass, as we have found no records where PA < AB. No extra investigation or validation is required here.

**R - Runs**

Runs should also be an integer value, in the range of 0 and some upper value. The upper value can be, at most, one larger than the number of plate appearances. In general, that's 12+1 for this dataset, but that's a highly unlikely value to see as number of runs. (We'll address individual records in the next step.)

In [24]:
print(pd.unique(df['R']))

[2 1 0 4 3 5 6]


**Note:** We want to do some validation within each individual data record to look for instances where the number of plate appearances is less than the number of runs (e.g., when a player pinch runs for a teammate and then scores a run they have 0 plate appearances and 1 run). And within that subset of records, look for instances where there is more than a difference of one between the `'PA'` and `'R'` values. (If we find any instances with a difference larger than one, we may have a data issue.)

In [25]:
records_pa_lt_r = df[df['PA'] < df['R']]
print("-----------------------------------------")
print("All records with PA < R")
print("-----------------------------------------")
print(records_pa_lt_r)

print("\n\n-----------------------------------------")
print("All records with PA < R where PA != R-1")
print("-----------------------------------------")
print(records_pa_lt_r[ records_pa_lt_r['PA'] != ((records_pa_lt_r['R']-1)) ])


-----------------------------------------
All records with PA < R
-----------------------------------------
                ID          Player   Tm  Opp  PA  AB  R  H  2B  3B  HR  RBI  \
97903     lowebo01      Bobby Lowe  DET  WSH   1   1  2  1   1   0   0  0.0   
203202   stanljo02     Joe Stanley  CHC  BSN   1   0  2  0   0   0   0  0.0   
221883   collibi02    Bill Collins  BSN  STL   1   1  2  1   0   0   0  NaN   
223671   keelewi01   Willie Keeler  NYG  CHC   1   1  2  1   0   0   0  NaN   
230698   butlear01      Art Butler  BSN  PHI   1   1  2  0   0   0   0  NaN   
...            ...             ...  ...  ...  ..  .. .. ..  ..  ..  ..  ...   
4245067  hamilbi02  Billy Hamilton  CHW  MIN   1   1  2  0   0   0   0  0.0   
4256455  dubonma01  Mauricio Dubon  SFG  PHI   1   1  2  1   0   0   0  0.0   
4262325  whiteel04       Eli White  TEX  OAK   1   1  2  0   0   0   0  0.0   
4264040   kempto01       Tony Kemp  OAK  LAA   1   1  2  1   0   0   0  0.0   
4268197  davisjo05  Jon

**Note:** The validation checks pass, as we have found no records where PA < R and PA != R-1. No extra investigation or validation is required here.

**H - Hits**

Hits should also be an integer value, in the range of 0 and some upper value. The upper value can be, at most, the number of plate appearances. In general, that's 12 for this dataset, but that's a highly unlikely value to see as number of hits. (We'll address individual records in the next step.)

In [26]:
pd.unique(df['H'])

array([2, 3, 1, 0, 4, 5, 6, 9, 7])

**Note:** We should also confirm that there are never more hits than plate appearances within individual records.

In [27]:
records_pa_lt_h = df[df['PA'] < df['H']]
print("-----------------------------------------")
print("All records with PA < H")
print("-----------------------------------------")
print(records_pa_lt_h)

-----------------------------------------
All records with PA < H
-----------------------------------------
Empty DataFrame
Columns: [ID, Player, Tm, Opp, PA, AB, R, H, 2B, 3B, HR, RBI, BB, SO, HBP, SH, SF, WPA, RE24, aLI, BOP, Result, Score, Season]
Index: []


**Note:** The validation checks pass, as we have found no records where PA < H. No extra investigation or validation is required here.

**2B - Doubles**
**3B - Triples**
**HR - Home Run**

All of these extra base hits must be integer values, within the range of 0 up to the number of Plate Appearances.

In [28]:
print("2B: ", pd.unique(df['2B']))
print("3B: ", pd.unique(df['3B']))
print("HR: ", pd.unique(df['HR']))

2B:  [1 0 2 4 3]
3B:  [0 1 3 2]
HR:  [0 1 2 3 4]


**Note:** These are all reasonable values at a glance.

We should also confirm that these values are never greater than the number of Plate Appearances, At Bats, or Hits within individual records. (Note that Hits (`'H'`) represents all kinds of hits, not just single-base hits.)

In [29]:
records_pa_lt_2b = df[df['PA'] < df['2B']]
print("-----------------------------------------")
print("All records with PA < 2B")
print("-----------------------------------------")
print(records_pa_lt_2b)


records_pa_lt_3b = df[df['PA'] < df['3B']]
print("\n-----------------------------------------")
print("All records with PA < 3B")
print("-----------------------------------------")
print(records_pa_lt_3b)


records_pa_lt_hr = df[df['PA'] < df['HR']]
print("\n-----------------------------------------")
print("All records with PA < HR")
print("-----------------------------------------")
print(records_pa_lt_hr)


records_ab_lt_2b = df[df['AB'] < df['2B']]
print("\n-----------------------------------------")
print("All records with AB < 2B")
print("-----------------------------------------")
print(records_ab_lt_2b)


records_ab_lt_3b = df[df['AB'] < df['3B']]
print("\n-----------------------------------------")
print("All records with AB < 3B")
print("-----------------------------------------")
print(records_ab_lt_3b)


records_ab_lt_hr = df[df['AB'] < df['HR']]
print("\n-----------------------------------------")
print("All records with AB < HR")
print("-----------------------------------------")
print(records_ab_lt_hr)


records_h_lt_2b = df[df['H'] < df['2B']]
print("\n-----------------------------------------")
print("All records with H < 2B")
print("-----------------------------------------")
print(records_h_lt_2b)


records_h_lt_3b = df[df['H'] < df['3B']]
print("\n-----------------------------------------")
print("All records with H < 3B")
print("-----------------------------------------")
print(records_h_lt_3b)


records_h_lt_hr = df[df['H'] < df['HR']]
print("\n-----------------------------------------")
print("All records with H < HR")
print("-----------------------------------------")
print(records_h_lt_hr)

-----------------------------------------
All records with PA < 2B
-----------------------------------------
                ID          Player   Tm  Opp  PA  AB  R  H  2B  3B  HR  RBI  \
1275367  robined01  Eddie Robinson  CHW  SLB   1   1  0  0   2   0   0  0.0   

         BB  SO  HBP  SH  SF  WPA  RE24  aLI  BOP Result Score  Season  
1275367   0   0    0   0 NaN  NaN   NaN  NaN    4      L  6-10    1950  

-----------------------------------------
All records with PA < 3B
-----------------------------------------
Empty DataFrame
Columns: [ID, Player, Tm, Opp, PA, AB, R, H, 2B, 3B, HR, RBI, BB, SO, HBP, SH, SF, WPA, RE24, aLI, BOP, Result, Score, Season]
Index: []

-----------------------------------------
All records with PA < HR
-----------------------------------------
Empty DataFrame
Columns: [ID, Player, Tm, Opp, PA, AB, R, H, 2B, 3B, HR, RBI, BB, SO, HBP, SH, SF, WPA, RE24, aLI, BOP, Result, Score, Season]
Index: []

-----------------------------------------
All records with 

**Note:** These validation checks caught a couple of problematic records. Looking at the entirety of these records, it looks like a simple data error, where the number of Hits needs to be updated to reflect the number of extra base hits. Instead of removing all records for these players, we will make these small data adjustments.

**First for Ed Robinson:**

In [30]:
ed_robinson = 1275367

df.loc[[ed_robinson]]

ID          Player   Tm  Opp  PA  AB  R  H  2B  3B  HR  RBI  \
1275367  robined01  Eddie Robinson  CHW  SLB   1   1  0  0   2   0   0  0.0   

         BB  SO  HBP  SH  SF  WPA  RE24  aLI  BOP Result Score  Season  
1275367   0   0    0   0 NaN  NaN   NaN  NaN    4      L  6-10    1950

For **Ed Robinson's record** (1275367), we will use the doubles (`'2B'`) statistic value as the value for Hits, Plate Appearances, and At Bats. These are reasonable guesses, based on the record.

In [31]:
df.at[ed_robinson, 'H'] = df.at[ed_robinson, '2B']
df.at[ed_robinson, 'AB'] = df.at[ed_robinson, '2B']
df.at[ed_robinson, 'PA'] = df.at[ed_robinson, '2B']

df.loc[[ed_robinson]]

ID          Player   Tm  Opp  PA  AB  R  H  2B  3B  HR  RBI  \
1275367  robined01  Eddie Robinson  CHW  SLB   2   2  0  2   2   0   0  0.0   

         BB  SO  HBP  SH  SF  WPA  RE24  aLI  BOP Result Score  Season  
1275367   0   0    0   0 NaN  NaN   NaN  NaN    4      L  6-10    1950

**Next for Joe Tipton:**

In [32]:
joe_tipton = 1272928

df.loc[[joe_tipton]]

ID      Player   Tm  Opp  PA  AB  R  H  2B  3B  HR  RBI  BB  \
1272928  tiptojo01  Joe Tipton  PHA  CHW   3   2  0  0   0   1   0  0.0   1   

         SO  HBP  SH  SF  WPA  RE24  aLI  BOP Result Score  Season  
1272928   1    0   0 NaN  NaN   NaN  NaN    8      L  3-10    1950

For **Joe Tipton's record** (1272928), we will use the triples (`'3B'`) statistic value as the value for Hits but it is possible that the statistics for Plate Appearances and At Bats is correct. Because it is not obviously wrong, we won't change these. These are reasonable guesses, based on the record.

In [33]:
df.at[joe_tipton, 'H'] = df.at[joe_tipton, '3B']

df.loc[[joe_tipton]]

ID      Player   Tm  Opp  PA  AB  R  H  2B  3B  HR  RBI  BB  \
1272928  tiptojo01  Joe Tipton  PHA  CHW   3   2  0  1   0   1   0  0.0   1   

         SO  HBP  SH  SF  WPA  RE24  aLI  BOP Result Score  Season  
1272928   1    0   0 NaN  NaN   NaN  NaN    8      L  3-10    1950

**Re-testing the original checks**

Re-running the tests that found these data issues should now pass and not introduce any new issues.

In [34]:
records_pa_lt_2b = df[df['PA'] < df['2B']]
print("-----------------------------------------")
print("All records with PA < 2B")
print("-----------------------------------------")
print(records_pa_lt_2b)


records_pa_lt_3b = df[df['PA'] < df['3B']]
print("\n-----------------------------------------")
print("All records with PA < 3B")
print("-----------------------------------------")
print(records_pa_lt_3b)


records_pa_lt_hr = df[df['PA'] < df['HR']]
print("\n-----------------------------------------")
print("All records with PA < HR")
print("-----------------------------------------")
print(records_pa_lt_hr)


records_ab_lt_2b = df[df['AB'] < df['2B']]
print("\n-----------------------------------------")
print("All records with AB < 2B")
print("-----------------------------------------")
print(records_ab_lt_2b)


records_ab_lt_3b = df[df['AB'] < df['3B']]
print("\n-----------------------------------------")
print("All records with AB < 3B")
print("-----------------------------------------")
print(records_ab_lt_3b)


records_ab_lt_hr = df[df['AB'] < df['HR']]
print("\n-----------------------------------------")
print("All records with AB < HR")
print("-----------------------------------------")
print(records_ab_lt_hr)


records_h_lt_2b = df[df['H'] < df['2B']]
print("\n-----------------------------------------")
print("All records with H < 2B")
print("-----------------------------------------")
print(records_h_lt_2b)


records_h_lt_3b = df[df['H'] < df['3B']]
print("\n-----------------------------------------")
print("All records with H < 3B")
print("-----------------------------------------")
print(records_h_lt_3b)


records_h_lt_hr = df[df['H'] < df['HR']]
print("\n-----------------------------------------")
print("All records with H < HR")
print("-----------------------------------------")
print(records_h_lt_hr)

-----------------------------------------
All records with PA < 2B
-----------------------------------------
Empty DataFrame
Columns: [ID, Player, Tm, Opp, PA, AB, R, H, 2B, 3B, HR, RBI, BB, SO, HBP, SH, SF, WPA, RE24, aLI, BOP, Result, Score, Season]
Index: []

-----------------------------------------
All records with PA < 3B
-----------------------------------------
Empty DataFrame
Columns: [ID, Player, Tm, Opp, PA, AB, R, H, 2B, 3B, HR, RBI, BB, SO, HBP, SH, SF, WPA, RE24, aLI, BOP, Result, Score, Season]
Index: []

-----------------------------------------
All records with PA < HR
-----------------------------------------
Empty DataFrame
Columns: [ID, Player, Tm, Opp, PA, AB, R, H, 2B, 3B, HR, RBI, BB, SO, HBP, SH, SF, WPA, RE24, aLI, BOP, Result, Score, Season]
Index: []

-----------------------------------------
All records with AB < 2B
-----------------------------------------
Empty DataFrame
Columns: [ID, Player, Tm, Opp, PA, AB, R, H, 2B, 3B, HR, RBI, BB, SO, HBP, SH, SF, WPA

**SUCCESS!!** These data issues have been resolved. `'2B'`, `'3B'`, and `'HR'` are now validated.

**RBI - RBIs (Runs Batted In)**

This is an important statistic, especially for calculated batting statistics which may prove useful later.

RBIs should be an integer value, in the range of 0 and some upper value. The upper value is dependent on the number of runners on base at the time of the plate appearance, which we do not know directly from the data. We can estimate a maximum possible value of Plate Appearances * 4 (the maximum number of runs possible to bat in). This maximum value would be statistically possible but highly unlikely. But it means that if we see a value higher than 12 * 4 = 48 in a single game then it is definitely out of range.


In [35]:
print(pd.unique(df['RBI']))

[ 1.  0.  3.  4.  2.  5.  6.  8.  7. nan  9. 12. 11. 10.]


**Note:** Of course, we don't see anything nearly as extreme as 48, but we have multiple problems here: (1) RBI is stored as a Float value, which makes no sense in this context, and (2) we have a NaN/undefined value to deal with.

While it would be nice to convert to integer values, the presence of NaN values blocks our carrying out this operation. So, first, we have to make a decision about how to deal with the NaN values.

Let's look at how many records include these NaN values and how many unique players are impacted by this data issue.

In [36]:
rbi_is_nan = df.loc[pd.isna(df['RBI'])]
print(rbi_is_nan)
print()

print("\nUnique Players with this RBI-NaN problem:")
rbi_nan_players = pd.unique(rbi_is_nan['ID'])
print(rbi_nan_players)
print("Count: ",len(rbi_nan_players))

print()
print("\nPercentage of overall records (",len(df),") impacted: ", len(rbi_is_nan)/len(df)*100)
print("There are",len(rbi_nan_players),"players impacted by these",len(rbi_is_nan),"records.")
print("")

               ID           Player   Tm  Opp  PA  AB  R  H  2B  3B  HR  RBI  \
41584   becklja01     Jake Beckley  CIN  PIT   4   4  2  2   0   0   0  NaN   
41594   corcoto01   Tommy Corcoran  CIN  PIT   4   4  0  0   0   0   0  NaN   
41598   donlimi01      Mike Donlin  CIN  PIT   4   4  1  1   0   0   0  NaN   
41615   kellejo01       Joe Kelley  CIN  PIT   4   3  0  1   0   0   0  NaN   
41621   magooge01    George Magoon  CIN  PIT   3   3  0  1   0   0   0  NaN   
...           ...              ...  ...  ...  ..  .. .. ..  ..  ..  ..  ...   
476024  wambsbi01  Bill Wambsganss  CLE  SLB   4   4  0  1   0   0   0  NaN   
476025  weavebu01      Buck Weaver  CHW  DET   5   4  1  3   0   2   0  NaN   
476027  wilkiro01    Roy Wilkinson  CHW  DET   3   3  1  1   1   0   0  NaN   
476029   woodjo02   Smoky Joe Wood  CLE  SLB   4   4  0  1   0   0   0  NaN   
476031  youngra01      Ralph Young  DET  CHW   5   4  0  2   0   0   0  NaN   

        BB  SO  HBP  SH  SF  WPA  RE24  aLI  BOP Re

While the number of overall records impacted is less than 1%, we see that there are many more players than there are records with this problem, which means that it is safest to remove all records for these players -- not only the problematic ones. Since all of the impacted data records are from seasons early in the 20th century, this does not raise any great concerns for this project.

In [37]:
# the removal
df = df[df['ID'].isin(rbi_nan_players) == False]

# test
print("\nTest for an empty list of RBI==NaN values, after removing rows:\n\n", df.loc[pd.isna(df['RBI'])])
print("\n\nReduced to",len(df),"records in main data frame after removing those players records entirely.")


Test for an empty list of RBI==NaN values, after removing rows:

 Empty DataFrame
Columns: [ID, Player, Tm, Opp, PA, AB, R, H, 2B, 3B, HR, RBI, BB, SO, HBP, SH, SF, WPA, RE24, aLI, BOP, Result, Score, Season]
Index: []


Reduced to 3715522 records in main data frame after removing those players records entirely.


**Finally,** we can address our other issue: converting the type for RBI from float to integer.

In [38]:
df['RBI'] = df['RBI'].astype(int)
# test
print(pd.unique(df['RBI']))

[ 0  1  3  2  4  5  6  7  8 12  9 11 10]


**SUCCESS!!** No NaN values and all Integer values. And all of these are reasonable values at a glance.

We should also confirm that RBI values are never greater than the number of Plate Appearances within individual records. (Note: There are many different scenarios for earning an RBI, but they all require a corresponding Plate Appearance.)

In [39]:
records_pa4_lt_rbi = df[df['PA']*4 < (df['RBI']) ]
print("\n-----------------------------------------")
print("All records with PA*4 < RBI")
print("-----------------------------------------")
print(records_pa4_lt_rbi)


-----------------------------------------
All records with PA*4 < RBI
-----------------------------------------
Empty DataFrame
Columns: [ID, Player, Tm, Opp, PA, AB, R, H, 2B, 3B, HR, RBI, BB, SO, HBP, SH, SF, WPA, RE24, aLI, BOP, Result, Score, Season]
Index: []


**Note:** All RBI-related data issues have been resolved and our check within a record has passed. `'RBI'` is now validated.

**BB - Base On Balls (Walks)**

Walks should be an integer value, within the range of 0 up to the number of Plate Appearances.

In [40]:
pd.unique(df['BB'])

array([1, 0, 2, 3, 4, 5, 6])

**Note:** These are all reasonable values at a glance.

We should also confirm that Walk values are never greater than the number of Plate Appearances within individual records.

In [41]:
records_pa_lt_bb = df[df['PA'] < (df['BB']) ]
print("\n-----------------------------------------")
print("All records with PA < BB")
print("-----------------------------------------")
print(records_pa_lt_bb)


-----------------------------------------
All records with PA < BB
-----------------------------------------
Empty DataFrame
Columns: [ID, Player, Tm, Opp, PA, AB, R, H, 2B, 3B, HR, RBI, BB, SO, HBP, SH, SF, WPA, RE24, aLI, BOP, Result, Score, Season]
Index: []


**HBP - Hit By Pitch**

Hit By Pitch should be an integer value, within the range of 0 up to the number of Plate Appearances.

Reference: https://en.wikipedia.org/wiki/Base_on_balls
HBP is **not** recorded as a walk/BB. ("A hit by pitch is not counted statistically as a walk, though the effect is mostly the same, with the batter receiving a free pass to first base.")

In [42]:
pd.unique(df['HBP'])

array([0, 1, 2, 3])

**Note:** These are all reasonable values at a glance.

We should also confirm that HBP values are never greater than the number of Plate Appearances within individual records.

In [43]:
records_pa_lt_hbp = df[df['PA'] < (df['HBP']) ]
print("\n-----------------------------------------")
print("All records with PA < HBP")
print("-----------------------------------------")
print(records_pa_lt_hbp)


-----------------------------------------
All records with PA < HBP
-----------------------------------------
Empty DataFrame
Columns: [ID, Player, Tm, Opp, PA, AB, R, H, 2B, 3B, HR, RBI, BB, SO, HBP, SH, SF, WPA, RE24, aLI, BOP, Result, Score, Season]
Index: []


**SO - Strikeouts**

Strikeouts should be an integer value, within the range of 0 up to the number of Plate Appearances.

In [44]:
print(pd.unique(df['SO']))

[0 2 1 3 4 5 6]


**Note:** These are all reasonable values at a glance.

We should also confirm that Strikeout values are never greater than the number of Plate Appearances or At Bats within individual records.

In [45]:
records_pa_lt_so = df[df['PA'] < (df['SO']) ]
print("\n-----------------------------------------")
print("All records with PA < SO")
print("-----------------------------------------")
print(records_pa_lt_so)

records_ab_lt_so = df[df['AB'] < (df['SO']) ]
print("\n-----------------------------------------")
print("All records with AB < SO")
print("-----------------------------------------")
print(records_ab_lt_so)


-----------------------------------------
All records with PA < SO
-----------------------------------------
Empty DataFrame
Columns: [ID, Player, Tm, Opp, PA, AB, R, H, 2B, 3B, HR, RBI, BB, SO, HBP, SH, SF, WPA, RE24, aLI, BOP, Result, Score, Season]
Index: []

-----------------------------------------
All records with AB < SO
-----------------------------------------
                ID       Player   Tm  Opp  PA  AB  R  H  2B  3B  HR  RBI  BB  \
1252967  frienow01  Owen Friend  SLB  NYY   2   0  1  0   0   0   0    0   1   

         SO  HBP  SH  SF  WPA  RE24  aLI  BOP Result Score  Season  
1252967   1    0   1 NaN  NaN   NaN  NaN    8      L  9-11    1950  


**Note:** We found one record with a strikeout but no at bats -- which doesn't make sense.

In [46]:
owen_friend = 1252967
df.loc[[owen_friend]]

ID       Player   Tm  Opp  PA  AB  R  H  2B  3B  HR  RBI  BB  \
1252967  frienow01  Owen Friend  SLB  NYY   2   0  1  0   0   0   0    0   1   

         SO  HBP  SH  SF  WPA  RE24  aLI  BOP Result Score  Season  
1252967   1    0   1 NaN  NaN   NaN  NaN    8      L  9-11    1950

Looking closer we can make a minor data adjustment for this record, and assume that there should be two At Bats instead of 0. In this game, the player stuck out once -- which means there was at least one At Bat.

Note: we also see a sacrifice hit (`'SH'`) listed. But, Sacrifice Hits and Flies (`'SF'`) do not count as times At Bat. They do, however, count as Plate Appearances.

These statistics demonstrate there must have been at least one At Bat. There was also one Walk (`'BB'`), which should mean there was at least three Plate Appearances, between the Sacrifice Hit, the Walk, and the Strikeout.

The most reasonable guess here is to set PA = 3 and AB = 1.

In [47]:
df.at[owen_friend, 'PA'] = 3
df.at[owen_friend, 'AB'] = 1

df.loc[[owen_friend]]

ID       Player   Tm  Opp  PA  AB  R  H  2B  3B  HR  RBI  BB  \
1252967  frienow01  Owen Friend  SLB  NYY   3   1  1  0   0   0   0    0   1   

         SO  HBP  SH  SF  WPA  RE24  aLI  BOP Result Score  Season  
1252967   1    0   1 NaN  NaN   NaN  NaN    8      L  9-11    1950

Re-Testing:

In [48]:
records_pa_lt_so = df[df['PA'] < (df['SO']) ]
print("\n-----------------------------------------")
print("All records with PA < SO")
print("-----------------------------------------")
print(records_pa_lt_so)

records_ab_lt_so = df[df['AB'] < (df['SO']) ]
print("\n-----------------------------------------")
print("All records with AB < SO")
print("-----------------------------------------")
print(records_ab_lt_so)


-----------------------------------------
All records with PA < SO
-----------------------------------------
Empty DataFrame
Columns: [ID, Player, Tm, Opp, PA, AB, R, H, 2B, 3B, HR, RBI, BB, SO, HBP, SH, SF, WPA, RE24, aLI, BOP, Result, Score, Season]
Index: []

-----------------------------------------
All records with AB < SO
-----------------------------------------
Empty DataFrame
Columns: [ID, Player, Tm, Opp, PA, AB, R, H, 2B, 3B, HR, RBI, BB, SO, HBP, SH, SF, WPA, RE24, aLI, BOP, Result, Score, Season]
Index: []


**SH - Sacrifice Hits**

Sacrifice Hits should be an integer value, within the range of 0 up to the number of Plate Appearances.

Note: Sacrifice plays do not count against the batter and, as such, don't count as At Bats.

References:
https://www.mlb.com/glossary/standard-stats/sacrifice-bunt
https://www.baseball-reference.com/bullpen/Sacrifice_hit

In [49]:
pd.unique(df['SH'])

array([0, 1, 2, 3, 4])

**Note:** These are all reasonable values at a glance.

We should also confirm that Sacrifce Hits values are never greater than the number of Plate Appearances within individual records.

In [50]:
records_pa_lt_sh = df[df['PA'] < (df['SH']) ]
print("\n-----------------------------------------")
print("All records with PA < SH")
print("-----------------------------------------")
print(records_pa_lt_sh)


-----------------------------------------
All records with PA < SH
-----------------------------------------
Empty DataFrame
Columns: [ID, Player, Tm, Opp, PA, AB, R, H, 2B, 3B, HR, RBI, BB, SO, HBP, SH, SF, WPA, RE24, aLI, BOP, Result, Score, Season]
Index: []


**SF - Sacrifice Flies**

Sacrifice Flies should be an integer value, within the range of 0 up to the number of Plate Appearances.

Note: Sacrifice plays do not count against the batter and, as such, don't count as At Bats.

Reference:
https://www.mlb.com/glossary/standard-stats/sacrifice-fly

In [51]:
pd.unique(df['SF'])

array([nan,  0.,  1.,  2.,  3.])

**Note:** We find that our Sacrifice Flies data is non-integer -- as with the RBI data processing, this is because there are NaN values present.

We will take the same approach for investigation, record removal, and conversion as with RBI.

In [52]:
sf_is_nan = df.loc[pd.isna(df['SF'])]
print(sf_is_nan)
print()

print("\nUnique Players with this SF-NaN problem:")
sf_nan_players = pd.unique(sf_is_nan['ID'])
print(sf_nan_players)
print("Count: ",len(sf_nan_players))

print()
print("\nPercentage of overall records (",len(df),") impacted: ", len(sf_is_nan)/len(df)*100)
print("There are",len(sf_nan_players),"players impacted by these",len(sf_is_nan),"records.")
print("")

                ID           Player   Tm  Opp  PA  AB  R  H  2B  3B  HR  RBI  \
4        delahed01     Ed Delahanty  PHI  BRO   5   4  1  2   0   0   0    0   
5        dolanjo02        Joe Dolan  PHI  BRO   5   5  0  1   0   0   0    1   
21       childcu01     Cupid Childs  CHC  STL   5   5  1  1   0   0   0    0   
22       crolifr01     Fred Crolius  BSN  NYG   4   4  0  0   0   0   0    1   
28       delahed01     Ed Delahanty  PHI  BRO   4   4  0  0   0   0   0    0   
...            ...              ...  ...  ...  ..  .. .. ..  ..  ..  ..  ...   
1355415  wyrosjo01  Johnny Wyrostek  PHI  BRO   4   4  0  1   0   0   0    0   
1355416   yosted01       Eddie Yost  WSH  PHA   5   3  1  2   1   0   0    1   
1355417  youngbo01      Bobby Young  SLB  CHW   4   4  0  0   0   0   0    0   
1355418  zernigu01      Gus Zernial  PHA  WSH   5   4  2  3   0   0   0    3   
1670929  bankser01      Ernie Banks  CHC  LAD   1   1  0  0   0   0   0    0   

         BB  SO  HBP  SH  SF    WPA   R

We see here that a much larger percentage of our dataset is impacted (21%), so before removing all of these records and all of the player data associated, we need to consider the potential importance of the Sacrifice Flies feature for the project.

Removing the SF column itself is not an option if we want to calculate the OBP (On Base Percentage) statistic.

Instead of NaN values, we will replace the NaN values with zeroes. It won't be accurate in all cases (since these haven't always been tracked), but the worst case will be that the affected records will underreport this statistic and consequently have a lower OBP percentage. But, it is the best we can do without completely removing the data.

In [87]:
series_of_sf = sf_is_nan.index
series_of_sf

df.loc[series_of_sf, 'SF'] = 0
df['SF'] = df['SF'].astype(int)
df

ID            Player   Tm  Opp  PA  AB  R  H  2B  3B  HR  RBI  \
4        delahed01      Ed Delahanty  PHI  BRO   5   4  1  2   0   0   0    0   
5        dolanjo02         Joe Dolan  PHI  BRO   5   5  0  1   0   0   0    1   
21       childcu01      Cupid Childs  CHC  STL   5   5  1  1   0   0   0    0   
22       crolifr01      Fred Crolius  BSN  NYG   4   4  0  0   0   0   0    1   
28       delahed01      Ed Delahanty  PHI  BRO   4   4  0  0   0   0   0    0   
...            ...               ...  ...  ...  ..  .. .. ..  ..  ..  ..  ...   
4285624  woodfja01     Jake Woodford  STL  CHC   2   1  0  0   0   0   0    0   
4285625  yastrmi01  Mike Yastrzemski  SFG  SDP   4   3  1  1   1   0   0    2   
4285626  zimmebr01    Bradley Zimmer  CLE  TEX   4   4  1  2   0   0   0    1   
4285627  zimmery01    Ryan Zimmerman  WSN  BOS   4   3  0  0   0   0   0    1   
4285628  zuninmi01       Mike Zunino  TBR  NYY   4   4  0  0   0   0   0    0   

         BB  SO  HBP  SH  SF Result  Season  
4         1   0    0   0   0      L    1901  
5         0   0    0   0   0      L    1901  
21        0   0    0   0   0      W    1901  
22        0   0    0   0   0      W    1901  
28        0   2    0   0   0      L    1901  
...      ..  ..  ...  ..  ..    ...     ...  
4285624   0   0    0   1   0      L    2021  
4285625   1   1    0   0   0      W    2021  
4285626   0   0    0   0   0      W    2021  
4285627   1   2    0   0   0      L    2021  
4285628   0   1    0   0   0      L    2021  

[3715522 rows x 19 columns]

In [86]:
pd.unique(df['SF'])

array([0, 1, 2, 3])

In [55]:
records_pa_lt_sf = df[df['PA'] < (df['SF']) ]
print("\n-----------------------------------------")
print("All records with PA < SF")
print("-----------------------------------------")
print(records_pa_lt_sf)


-----------------------------------------
All records with PA < SF
-----------------------------------------
Empty DataFrame
Columns: [ID, Player, Tm, Opp, PA, AB, R, H, 2B, 3B, HR, RBI, BB, SO, HBP, SH, SF, WPA, RE24, aLI, BOP, Result, Score, Season]
Index: []


**WPA - Win Probability Added**


Reference:
https://en.wikipedia.org/wiki/Win_probability_added

In [56]:
pd.unique(df['WPA'])

array([   nan,  0.03 , -0.042, ...,  0.974,  0.947, -0.655])

In [57]:
wpa_is_nan = df.loc[pd.isna(df['WPA'])]
print(wpa_is_nan)
print()

print("\nUnique Players with this SF-NaN problem:")
wpa_nan_players = pd.unique(wpa_is_nan['ID'])
print(wpa_nan_players)
print("Count: ",len(wpa_nan_players))

print()
print("\nPercentage of overall records (",len(df),") impacted: ", len(wpa_is_nan)/len(df)*100)
print("There are",len(wpa_nan_players),"players impacted by these",len(wpa_is_nan),"records.")
print("")

                ID         Player   Tm  Opp  PA  AB  R  H  2B  3B  HR  RBI  \
4        delahed01   Ed Delahanty  PHI  BRO   5   4  1  2   0   0   0    0   
5        dolanjo02      Joe Dolan  PHI  BRO   5   5  0  1   0   0   0    1   
21       childcu01   Cupid Childs  CHC  STL   5   5  1  1   0   0   0    0   
22       crolifr01   Fred Crolius  BSN  NYG   4   4  0  0   0   0   0    1   
28       delahed01   Ed Delahanty  PHI  BRO   4   4  0  0   0   0   0    0   
...            ...            ...  ...  ...  ..  .. .. ..  ..  ..  ..  ...   
1981210  raderdo02     Doug Rader  HOU  ATL   4   4  1  2   0   0   1    1   
1981254  watsobo01     Bob Watson  HOU  ATL   4   4  0  1   0   0   0    1   
1981257  williea02  Earl Williams  ATL  HOU   4   4  0  2   0   0   0    1   
1981259  wilsodo01     Don Wilson  HOU  ATL   3   2  0  0   0   0   0    0   
1981262   wynnji01       Jim Wynn  HOU  ATL   5   4  1  1   0   0   0    1   

         BB  SO  HBP  SH  SF  WPA  RE24  aLI  BOP Result Score 

**RE24 - Base-Out Runs Added**
This statistic represents the run expectancy based on 24 base outs.

Reference:
https://thebaseballscholar.com/2017/08/14/sabermetrics-101-re24/

In [58]:
pd.unique(df['RE24'])

array([   nan,  1.036, -0.138, ...,  5.928, -3.176,  5.142])

In [59]:
re24_is_nan = df.loc[pd.isna(df['RE24'])]
print(re24_is_nan)
print()

print("\nUnique Players with this SF-NaN problem:")
re24_nan_players = pd.unique(re24_is_nan['ID'])
print(re24_nan_players)
print("Count: ",len(re24_nan_players))

print()
print("\nPercentage of overall records (",len(df),") impacted: ", len(re24_is_nan)/len(df)*100)
print("There are",len(re24_nan_players),"players impacted by these",len(re24_is_nan),"records.")
print("")

                ID         Player   Tm  Opp  PA  AB  R  H  2B  3B  HR  RBI  \
4        delahed01   Ed Delahanty  PHI  BRO   5   4  1  2   0   0   0    0   
5        dolanjo02      Joe Dolan  PHI  BRO   5   5  0  1   0   0   0    1   
21       childcu01   Cupid Childs  CHC  STL   5   5  1  1   0   0   0    0   
22       crolifr01   Fred Crolius  BSN  NYG   4   4  0  0   0   0   0    1   
28       delahed01   Ed Delahanty  PHI  BRO   4   4  0  0   0   0   0    0   
...            ...            ...  ...  ...  ..  .. .. ..  ..  ..  ..  ...   
1981210  raderdo02     Doug Rader  HOU  ATL   4   4  1  2   0   0   1    1   
1981254  watsobo01     Bob Watson  HOU  ATL   4   4  0  1   0   0   0    1   
1981257  williea02  Earl Williams  ATL  HOU   4   4  0  2   0   0   0    1   
1981259  wilsodo01     Don Wilson  HOU  ATL   3   2  0  0   0   0   0    0   
1981262   wynnji01       Jim Wynn  HOU  ATL   5   4  1  1   0   0   0    1   

         BB  SO  HBP  SH  SF  WPA  RE24  aLI  BOP Result Score 

**aLI - Average Leverage Index**

Reference:
https://www.azsnakepit.com/2021/11/9/22742763/what-average-leverage-index-revealed-about-the-2021-diamondbacks

In [60]:
pd.unique(df['aLI'])

array([  nan, 0.14 , 1.187, ..., 4.373, 3.954, 7.45 ])

In [61]:
ali_is_nan = df.loc[pd.isna(df['aLI'])]
print(ali_is_nan)
print()

print("\nUnique Players with this SF-NaN problem:")
ali_nan_players = pd.unique(ali_is_nan['ID'])
print(ali_nan_players)
print("Count: ",len(ali_nan_players))

print()
print("\nPercentage of overall records (",len(df),") impacted: ", len(ali_is_nan)/len(df)*100)
print("There are",len(ali_nan_players),"players impacted by these",len(ali_is_nan),"records.")
print("")

                ID         Player   Tm  Opp  PA  AB  R  H  2B  3B  HR  RBI  \
4        delahed01   Ed Delahanty  PHI  BRO   5   4  1  2   0   0   0    0   
5        dolanjo02      Joe Dolan  PHI  BRO   5   5  0  1   0   0   0    1   
21       childcu01   Cupid Childs  CHC  STL   5   5  1  1   0   0   0    0   
22       crolifr01   Fred Crolius  BSN  NYG   4   4  0  0   0   0   0    1   
28       delahed01   Ed Delahanty  PHI  BRO   4   4  0  0   0   0   0    0   
...            ...            ...  ...  ...  ..  .. .. ..  ..  ..  ..  ...   
1981210  raderdo02     Doug Rader  HOU  ATL   4   4  1  2   0   0   1    1   
1981254  watsobo01     Bob Watson  HOU  ATL   4   4  0  1   0   0   0    1   
1981257  williea02  Earl Williams  ATL  HOU   4   4  0  2   0   0   0    1   
1981259  wilsodo01     Don Wilson  HOU  ATL   3   2  0  0   0   0   0    0   
1981262   wynnji01       Jim Wynn  HOU  ATL   5   4  1  1   0   0   0    1   

         BB  SO  HBP  SH  SF  WPA  RE24  aLI  BOP Result Score 

**Note:** An interesting phenomenon was seen with the WPA, RE24, and aLI statistics. They all seem to have been calculated in a similar range of time.

***A DECISION ... FOR WPA, RE24, and aLI STATISTICS***

After looking at all of this data, the nature of the statistics, and the goals of this project, the best decision is to remove these three features entirely. These three statistics are quite advanced to a point that is beyond the scope of this project.

So, we will remove these columns from the data frame.

In [62]:
del df['WPA']
del df['RE24']
del df['aLI']

In [63]:
df

ID            Player   Tm  Opp  PA  AB  R  H  2B  3B  HR  RBI  \
4        delahed01      Ed Delahanty  PHI  BRO   5   4  1  2   0   0   0    0   
5        dolanjo02         Joe Dolan  PHI  BRO   5   5  0  1   0   0   0    1   
21       childcu01      Cupid Childs  CHC  STL   5   5  1  1   0   0   0    0   
22       crolifr01      Fred Crolius  BSN  NYG   4   4  0  0   0   0   0    1   
28       delahed01      Ed Delahanty  PHI  BRO   4   4  0  0   0   0   0    0   
...            ...               ...  ...  ...  ..  .. .. ..  ..  ..  ..  ...   
4285624  woodfja01     Jake Woodford  STL  CHC   2   1  0  0   0   0   0    0   
4285625  yastrmi01  Mike Yastrzemski  SFG  SDP   4   3  1  1   1   0   0    2   
4285626  zimmebr01    Bradley Zimmer  CLE  TEX   4   4  1  2   0   0   0    1   
4285627  zimmery01    Ryan Zimmerman  WSN  BOS   4   3  0  0   0   0   0    1   
4285628  zuninmi01       Mike Zunino  TBR  NYY   4   4  0  0   0   0   0    0   

         BB  SO  HBP  SH  SF  BOP Result Score  Season  
4         1   0    0   0   0    3      L  7-12    1901  
5         0   0    0   0   0    7      L  7-12    1901  
21        0   0    0   0   0    1      W   8-7    1901  
22        0   0    0   0   0    6      W   7-0    1901  
28        0   2    0   0   0    3      L  2-10    1901  
...      ..  ..  ...  ..  ..  ...    ...   ...     ...  
4285624   0   0    0   1   0    9      L   2-3    2021  
4285625   1   1    0   0   0    7      W  11-4    2021  
4285626   0   0    0   0   0    6      W   6-0    2021  
4285627   1   2    0   0   0    5      L   5-7    2021  
4285628   0   1    0   0   0    7      L   0-1    2021  

[3715522 rows x 21 columns]

Considering what remains, `'BOP'` (Batting Order Position) and the extracted `'Score'`, I do not foresee any use for either of these, so I will remove these columns as well.

At this time, I have decided to keep the `'Tm'` and `'Opp'` columns in the data for context but do not anticipate using it in any of the learning aspects of the project.

In [64]:
del df['BOP']
del df['Score']

In [65]:
df

ID            Player   Tm  Opp  PA  AB  R  H  2B  3B  HR  RBI  \
4        delahed01      Ed Delahanty  PHI  BRO   5   4  1  2   0   0   0    0   
5        dolanjo02         Joe Dolan  PHI  BRO   5   5  0  1   0   0   0    1   
21       childcu01      Cupid Childs  CHC  STL   5   5  1  1   0   0   0    0   
22       crolifr01      Fred Crolius  BSN  NYG   4   4  0  0   0   0   0    1   
28       delahed01      Ed Delahanty  PHI  BRO   4   4  0  0   0   0   0    0   
...            ...               ...  ...  ...  ..  .. .. ..  ..  ..  ..  ...   
4285624  woodfja01     Jake Woodford  STL  CHC   2   1  0  0   0   0   0    0   
4285625  yastrmi01  Mike Yastrzemski  SFG  SDP   4   3  1  1   1   0   0    2   
4285626  zimmebr01    Bradley Zimmer  CLE  TEX   4   4  1  2   0   0   0    1   
4285627  zimmery01    Ryan Zimmerman  WSN  BOS   4   3  0  0   0   0   0    1   
4285628  zuninmi01       Mike Zunino  TBR  NYY   4   4  0  0   0   0   0    0   

         BB  SO  HBP  SH  SF Result  Season  
4         1   0    0   0   0      L    1901  
5         0   0    0   0   0      L    1901  
21        0   0    0   0   0      W    1901  
22        0   0    0   0   0      W    1901  
28        0   2    0   0   0      L    1901  
...      ..  ..  ...  ..  ..    ...     ...  
4285624   0   0    0   1   0      L    2021  
4285625   1   1    0   0   0      W    2021  
4285626   0   0    0   0   0      W    2021  
4285627   1   2    0   0   0      L    2021  
4285628   0   1    0   0   0      L    2021  

[3715522 rows x 19 columns]

**Result**

I am intentionally keeping `'Result'` at this time, so that I have options to understand how many winning games a player participated in, although there is a good chance this level of nuance will end up being outside of the scope of the project.

These values are extracted strings and should be one of 3 values: "W", "L", or "T". (Note that "T" represents a tied game, which is relatively unusual in baseball.)

In [66]:
print(pd.unique(df['Result']))

['L' 'W' 'T']


We might decide later to convert this to numeric values, if it proves to be helpful.

### Looking at Player Data

So far, we've looked at data from the perspective of each individual game played by an individual player. Now the data is healthier than when we started, we want to look at individual players during their careers.

Keeping in mind that our goal is to find trends that might predict a batter's success in the future, we will want to be able to combine their statistics over time. It might also be useful to attempt to find ways to examine their data during parts of their careers, but that's not currently a focus

In [67]:
print("There are",len(pd.unique(df['ID'])),"unique players remaining in this dataset.")

There are 14336 unique players remaining in this dataset.


Let's try splitting the data into these arbitrary categories, somewhat by era - chosen primarily based on my own seasons of personal interest. The latter category might interesting for predictions, largely as unseen data that includes many current day players.

In [68]:
df_early = df[df['Season'] < 1977].copy()

In [69]:
df_early

ID           Player   Tm  Opp  PA  AB  R  H  2B  3B  HR  RBI  \
4        delahed01     Ed Delahanty  PHI  BRO   5   4  1  2   0   0   0    0   
5        dolanjo02        Joe Dolan  PHI  BRO   5   5  0  1   0   0   0    1   
21       childcu01     Cupid Childs  CHC  STL   5   5  1  1   0   0   0    0   
22       crolifr01     Fred Crolius  BSN  NYG   4   4  0  0   0   0   0    1   
28       delahed01     Ed Delahanty  PHI  BRO   4   4  0  0   0   0   0    0   
...            ...              ...  ...  ...  ..  .. .. ..  ..  ..  ..  ...   
2150289  rudolke01      Ken Rudolph  STL  PIT   3   3  0  0   0   0   0    0   
2150290  stennre01  Rennie Stennett  PIT  STL   4   4  0  0   0   0   0    0   
2150291  taverfr01    Frank Taveras  PIT  STL   1   1  0  0   0   0   0    0   
2150292  templga01  Garry Templeton  STL  PIT   2   2  0  0   0   0   0    0   
2150293   ziskri01      Richie Zisk  PIT  STL   4   3  0  1   1   0   0    0   

         BB  SO  HBP  SH  SF Result  Season  
4         1   0    0   0   0      L    1901  
5         0   0    0   0   0      L    1901  
21        0   0    0   0   0      W    1901  
22        0   0    0   0   0      W    1901  
28        0   2    0   0   0      L    1901  
...      ..  ..  ...  ..  ..    ...     ...  
2150289   0   0    0   0   0      L    1976  
2150290   0   0    0   0   0      W    1976  
2150291   0   1    0   0   0      W    1976  
2150292   0   0    0   0   0      L    1976  
2150293   1   0    0   0   0      W    1976  

[1580187 rows x 19 columns]

In [176]:
df_mid = df[df['Season'] > 1976].copy()

In [177]:
df_mid = df_mid[df_mid['Season'] < 2000].copy()

In [178]:
df_mid

ID         Player   Tm  Opp  PA  AB  R  H  2B  3B  HR  RBI  \
2150294  almonbi01     Bill Almon  SDP  CIN   4   4  0  0   0   0   0    0   
2150295  armbred01  Ed Armbrister  CIN  SDP   1   1  0  0   0   0   0    0   
2150296   baezjo01      Jose Baez  SEA  CAL   4   4  0  2   0   0   0    0   
2150297  baylodo01     Don Baylor  CAL  SEA   5   3  1  1   1   0   0    1   
2150298  bochtbr01   Bruce Bochte  CAL  SEA   5   3  2  2   0   0   0    0   
...            ...            ...  ...  ...  ..  .. .. ..  ..  ..  ..  ...   
3187645  stynech01   Chris Stynes  CIN  NYM   1   1  0  0   0   0   0    0   
3187646  taubeed01   Ed Taubensee  CIN  NYM   3   2  0  0   0   0   0    0   
3187647  vaughgr01    Greg Vaughn  CIN  NYM   4   3  0  0   0   0   0    0   
3187648  venturo01  Robin Ventura  NYM  CIN   5   3  0  1   0   0   0    1   
3187649  youngdm01   Dmitri Young  CIN  NYM   4   4  0  0   0   0   0    0   

         BB  SO  HBP  SH  SF Result  Season  
2150294   0   1    0   0   0      L    1977  
2150295   0   0    0   0   0      W    1977  
2150296   0   0    0   0   0      L    1977  
2150297   2   0    0   0   0      W    1977  
2150298   2   0    0   0   0      W    1977  
...      ..  ..  ...  ..  ..    ...     ...  
3187645   0   0    0   0   0      L    1999  
3187646   1   0    0   0   0      L    1999  
3187647   1   2    0   0   0      L    1999  
3187648   2   1    0   0   0      W    1999  
3187649   0   0    0   0   0      L    1999  

[1037356 rows x 19 columns]

In [179]:
df_later = df[df['Season'] > 1999].copy()

In [180]:
df_later

ID            Player   Tm  Opp  PA  AB  R  H  2B  3B  HR  RBI  \
3187650  alfoned01   Edgardo Alfonzo  NYM  CHC   4   3  1  1   1   0   0    0   
3187651  andresh01     Shane Andrews  CHC  NYM   5   3  1  2   0   0   1    3   
3187652   bellde01        Derek Bell  NYM  CHC   4   4  0  1   0   0   0    0   
3187653  brockta01      Tarrik Brock  CHC  NYM   1   1  0  1   0   0   0    0   
3187654  buforda01      Damon Buford  CHC  NYM   5   3  1  2   0   0   0    1   
...            ...               ...  ...  ...  ..  .. .. ..  ..  ..  ..  ...   
4285624  woodfja01     Jake Woodford  STL  CHC   2   1  0  0   0   0   0    0   
4285625  yastrmi01  Mike Yastrzemski  SFG  SDP   4   3  1  1   1   0   0    2   
4285626  zimmebr01    Bradley Zimmer  CLE  TEX   4   4  1  2   0   0   0    1   
4285627  zimmery01    Ryan Zimmerman  WSN  BOS   4   3  0  0   0   0   0    1   
4285628  zuninmi01       Mike Zunino  TBR  NYY   4   4  0  0   0   0   0    0   

         BB  SO  HBP  SH  SF Result  Season  
3187650   1   1    0   0   0      L    2000  
3187651   2   1    0   0   0      W    2000  
3187652   0   0    0   0   0      L    2000  
3187653   0   0    0   0   0      W    2000  
3187654   2   0    0   0   0      W    2000  
...      ..  ..  ...  ..  ..    ...     ...  
4285624   0   0    0   1   0      L    2021  
4285625   1   1    0   0   0      W    2021  
4285626   0   0    0   0   0      W    2021  
4285627   1   2    0   0   0      L    2021  
4285628   0   1    0   0   0      L    2021  

[1097979 rows x 19 columns]

In [181]:
df_mid

ID         Player   Tm  Opp  PA  AB  R  H  2B  3B  HR  RBI  \
2150294  almonbi01     Bill Almon  SDP  CIN   4   4  0  0   0   0   0    0   
2150295  armbred01  Ed Armbrister  CIN  SDP   1   1  0  0   0   0   0    0   
2150296   baezjo01      Jose Baez  SEA  CAL   4   4  0  2   0   0   0    0   
2150297  baylodo01     Don Baylor  CAL  SEA   5   3  1  1   1   0   0    1   
2150298  bochtbr01   Bruce Bochte  CAL  SEA   5   3  2  2   0   0   0    0   
...            ...            ...  ...  ...  ..  .. .. ..  ..  ..  ..  ...   
3187645  stynech01   Chris Stynes  CIN  NYM   1   1  0  0   0   0   0    0   
3187646  taubeed01   Ed Taubensee  CIN  NYM   3   2  0  0   0   0   0    0   
3187647  vaughgr01    Greg Vaughn  CIN  NYM   4   3  0  0   0   0   0    0   
3187648  venturo01  Robin Ventura  NYM  CIN   5   3  0  1   0   0   0    1   
3187649  youngdm01   Dmitri Young  CIN  NYM   4   4  0  0   0   0   0    0   

         BB  SO  HBP  SH  SF Result  Season  
2150294   0   1    0   0   0      L    1977  
2150295   0   0    0   0   0      W    1977  
2150296   0   0    0   0   0      L    1977  
2150297   2   0    0   0   0      W    1977  
2150298   2   0    0   0   0      W    1977  
...      ..  ..  ...  ..  ..    ...     ...  
3187645   0   0    0   0   0      L    1999  
3187646   1   0    0   0   0      L    1999  
3187647   1   2    0   0   0      L    1999  
3187648   2   1    0   0   0      W    1999  
3187649   0   0    0   0   0      L    1999  

[1037356 rows x 19 columns]

In [76]:
### TEST -- but if successful might do it for the entire df for career/lifetime stats as a record

temp = df_mid.groupby('ID')
temp

In [88]:
summables = ['PA','AB','R','H','2B','3B','HR','RBI','BB','SO','HBP','SH', 'SF']
temp_mids = temp[summables].sum()
temp_mids

PA   AB   R    H  2B  3B  HR  RBI  BB   SO  HBP  SH  SF
ID                                                                 
aardsda01    4    3   0    0   0   0   0    0   0    1    0   1   0
aasedo01     5    5   0    0   0   0   0    0   0    3    0   0   0
abadan01    25   21   1    2   0   0   0    0   4    5    0   0   0
abadfe01     1    1   0    0   0   0   0    0   0    1    0   0   0
abbotje01  651  596  82  157  33   2  18   83  38   91    3   5   7
...        ...  ...  ..  ...  ..  ..  ..  ...  ..  ...  ...  ..  ..
zoskyed01   53   50   4    8   1   2   0    3   1   13    0   1   1
zuberjo01  151  136  13   34   7   1   3   16  12   20    1   1   1
zuletju01  191  174  23   43  11   0   9   36  10   51    6   0   1
zupcibo01  886  795  93  199  47   4   7   80  57  137    6  20   8
zuvelpa01  545  491  40  109  17   2   2   20  34   50    2  18   0

[5825 rows x 13 columns]

In [79]:
temp_player_names = df_mid[['Player','ID']]
temp_player_names

Player         ID
2150294      Bill Almon  almonbi01
2150295   Ed Armbrister  armbred01
2150296       Jose Baez   baezjo01
2150297      Don Baylor  baylodo01
2150298    Bruce Bochte  bochtbr01
...                 ...        ...
3805700     Chris Young  youngch04
3805701    Delmon Young  youngde03
3805702  Eric Young Jr.  younger03
3805703   Michael Young  youngmi02
3805704     Ben Zobrist  zobribe01

[1655411 rows x 2 columns]

In [182]:
temp_mids

PA   AB   R    H  2B  3B  HR  RBI  BB   SO  HBP  SH  SF
ID                                                                 
aardsda01    4    3   0    0   0   0   0    0   0    1    0   1   0
aasedo01     5    5   0    0   0   0   0    0   0    3    0   0   0
abadan01    25   21   1    2   0   0   0    0   4    5    0   0   0
abadfe01     1    1   0    0   0   0   0    0   0    1    0   0   0
abbotje01  651  596  82  157  33   2  18   83  38   91    3   5   7
...        ...  ...  ..  ...  ..  ..  ..  ...  ..  ...  ...  ..  ..
zoskyed01   53   50   4    8   1   2   0    3   1   13    0   1   1
zuberjo01  151  136  13   34   7   1   3   16  12   20    1   1   1
zuletju01  191  174  23   43  11   0   9   36  10   51    6   0   1
zupcibo01  886  795  93  199  47   4   7   80  57  137    6  20   8
zuvelpa01  545  491  40  109  17   2   2   20  34   50    2  18   0

[5825 rows x 13 columns]

In [204]:
temp_merged_stats = pd.merge(temp_mids, temp_player_names, on = "ID", how = "right")

In [199]:
columns = ['ID','Player'] + summables
columns

['ID',
 'Player',
 'PA',
 'AB',
 'R',
 'H',
 '2B',
 '3B',
 'HR',
 'RBI',
 'BB',
 'SO',
 'HBP',
 'SH',
 'SF']

In [205]:
temp_merged_stats = temp_merged_stats[columns]
temp_merged_stats

ID          Player    PA    AB     R     H   2B  3B   HR  RBI  \
0        almonbi01      Bill Almon  3549  3225   366   816  134  25   35  287   
1        armbred01   Ed Armbrister    91    78     9    20    4   3    1    5   
2         baezjo01       Jose Baez   391   355    43    87   14   2    1   19   
3        baylodo01      Don Baylor  6722  5846   904  1506  265  13  266  979   
4        bochtbr01    Bruce Bochte  4802  4196   525  1198  210  16   90  535   
...            ...             ...   ...   ...   ...   ...  ...  ..  ...  ...   
1655406  youngch04     Chris Young  3225  2848   417   683  174  17  118  367   
1655407  youngde03    Delmon Young  2967  2784   342   802  158   9   71  408   
1655408  younger03  Eric Young Jr.   479   427    67   105   10   4    1   19   
1655409  youngmi02   Michael Young  7396  6788  1005  2061  388  52  169  917   
1655410  zobribe01     Ben Zobrist  2458  2108   316   542  120  19   72  314   

          BB    SO  HBP  SH  SF  
0        246   617    6  43  29  
1         10    21    0   2   1  
2         25    27    1  10   0  
3        583   775  201   6  86  
4        526   543    8  31  41  
...      ...   ...  ...  ..  ..  
1655406  324   735   18  12  23  
1655407  125   514   25   1  32  
1655408   47    82    3   1   1  
1655409  499  1082   20  25  64  
1655410  298   423   10  14  28  

[1655411 rows x 15 columns]

In [210]:
temp_merged_stats['AVG'] = temp_merged_stats['H'] / (temp_merged_stats['AB']*1.0)

temp_merged_stats['SLG'] = (temp_merged_stats['H'] + temp_merged_stats['2B'] + 2*temp_merged_stats['3B'] + 3*temp_merged_stats['HR']) / (temp_merged_stats['AB']*1.0)

temp_merged_stats['OBP'] = (temp_merged_stats['H'] + temp_merged_stats['BB'] + temp_merged_stats['HBP']) / ((temp_merged_stats['AB'] + temp_merged_stats['BB'] + temp_merged_stats['HBP'] + temp_merged_stats['SF'])*1.0) 

temp_merged_stats['OPS'] = temp_merged_stats['SLG'] + temp_merged_stats['OBP']

temp_merged_stats['RC'] = (temp_merged_stats['H'] + temp_merged_stats['2B'] + 2*temp_merged_stats['3B'] + 3*temp_merged_stats['HR']) * ( (temp_merged_stats['H'] + temp_merged_stats['BB']) / (temp_merged_stats['AB'] + temp_merged_stats['BB']) )

temp_merged_stats['ISO'] = (temp_merged_stats['2B'] + 2*temp_merged_stats['3B'] + 3*temp_merged_stats['HR']) / (temp_merged_stats['AB']*1.0)


temp_merged_stats

ID          Player    PA    AB     R     H   2B  3B   HR  RBI  \
0        almonbi01      Bill Almon  3549  3225   366   816  134  25   35  287   
1        armbred01   Ed Armbrister    91    78     9    20    4   3    1    5   
2         baezjo01       Jose Baez   391   355    43    87   14   2    1   19   
3        baylodo01      Don Baylor  6722  5846   904  1506  265  13  266  979   
4        bochtbr01    Bruce Bochte  4802  4196   525  1198  210  16   90  535   
...            ...             ...   ...   ...   ...   ...  ...  ..  ...  ...   
1655406  youngch04     Chris Young  3225  2848   417   683  174  17  118  367   
1655407  youngde03    Delmon Young  2967  2784   342   802  158   9   71  408   
1655408  younger03  Eric Young Jr.   479   427    67   105   10   4    1   19   
1655409  youngmi02   Michael Young  7396  6788  1005  2061  388  52  169  917   
1655410  zobribe01     Ben Zobrist  2458  2108   316   542  120  19   72  314   

          BB    SO  HBP  SH  SF       AVG       SLG       OBP       OPS  \
0        246   617    6  43  29  0.253023  0.342636  0.304621  0.647256   
1         10    21    0   2   1  0.256410  0.423077  0.337079  0.760156   
2         25    27    1  10   0  0.245070  0.304225  0.296588  0.600813   
3        583   775  201   6  86  0.257612  0.443893  0.340977  0.784870   
4        526   543    8  31  41  0.285510  0.407531  0.363027  0.770558   
...      ...   ...  ...  ..  ..       ...       ...       ...       ...   
1655406  324   735   18  12  23  0.239817  0.437149  0.319016  0.756165   
1655407  125   514   25   1  32  0.288075  0.427802  0.320971  0.748773   
1655408   47    82    3   1   1  0.245902  0.295082  0.324268  0.619350   
1655409  499  1082   20  25  64  0.303624  0.450796  0.350020  0.800816   
1655410  298   423   10  14  28  0.257116  0.434535  0.347791  0.782326   

                  RC       ISO      PA_SO      PA_BB     SO_BB     PA_RBI  
0         338.089888  0.089612   5.752026  14.426829  2.508130  12.365854  
1          11.250000  0.166667   4.333333   9.100000  2.100000  18.200000  
2          31.831579  0.059155  14.481481  15.640000  1.080000  20.578947  
3         843.203453  0.186281   8.673548  11.530017  1.329331   6.866190  
4         624.320203  0.122021   8.843462   9.129278  1.032319   8.975701  
...              ...       ...        ...        ...       ...        ...  
1655406   395.244325  0.197331   4.387755   9.953704  2.268519   8.787466  
1655407   379.531454  0.139727   5.772374  23.736000  4.112000   7.272059  
1655408    40.405063  0.049180   5.841463  10.191489  1.744681  25.210526  
1655409  1075.010292  0.147171   6.835490  14.821643  2.168337   8.065431  
1655410   319.800499  0.177419   5.810875   8.248322  1.419463   7.828025  

[1655411 rows x 25 columns]

In [290]:
temp_prepped_stats = temp_merged_stats[['ID', 'Player', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'SO', 'AVG', 'SLG', 'OBP', 'OPS']]
temp_prepped_stats

ID          Player    PA    AB     R     H   2B  3B   HR  RBI  \
0        almonbi01      Bill Almon  3549  3225   366   816  134  25   35  287   
1        armbred01   Ed Armbrister    91    78     9    20    4   3    1    5   
2         baezjo01       Jose Baez   391   355    43    87   14   2    1   19   
3        baylodo01      Don Baylor  6722  5846   904  1506  265  13  266  979   
4        bochtbr01    Bruce Bochte  4802  4196   525  1198  210  16   90  535   
...            ...             ...   ...   ...   ...   ...  ...  ..  ...  ...   
1655406  youngch04     Chris Young  3225  2848   417   683  174  17  118  367   
1655407  youngde03    Delmon Young  2967  2784   342   802  158   9   71  408   
1655408  younger03  Eric Young Jr.   479   427    67   105   10   4    1   19   
1655409  youngmi02   Michael Young  7396  6788  1005  2061  388  52  169  917   
1655410  zobribe01     Ben Zobrist  2458  2108   316   542  120  19   72  314   

          BB    SO       AVG       SLG       OBP       OPS  
0        246   617  0.253023  0.342636  0.304621  0.647256  
1         10    21  0.256410  0.423077  0.337079  0.760156  
2         25    27  0.245070  0.304225  0.296588  0.600813  
3        583   775  0.257612  0.443893  0.340977  0.784870  
4        526   543  0.285510  0.407531  0.363027  0.770558  
...      ...   ...       ...       ...       ...       ...  
1655406  324   735  0.239817  0.437149  0.319016  0.756165  
1655407  125   514  0.288075  0.427802  0.320971  0.748773  
1655408   47    82  0.245902  0.295082  0.324268  0.619350  
1655409  499  1082  0.303624  0.450796  0.350020  0.800816  
1655410  298   423  0.257116  0.434535  0.347791  0.782326  

[1655411 rows x 16 columns]

### Getting the Supplemental Hall of Fame Data

In [291]:
HoF_data = pd.read_csv('./data/halloffamers.csv')
HoF_data

Year                         Name   Unnamed: 2  Voted By  \
0    2022         Bud Fowler\fowlebu99    1858-1913  Veterans   
1    2022         Gil Hodges\hodgegi01    1924-1972  Veterans   
2    2022            Jim Kaat\kaatji01  1938-Living  Veterans   
3    2022      Minnie Minoso\minosmi01    1923-2015  Veterans   
4    2022        Buck O'Neil\oneilbu01    1911-2006  Veterans   
..    ...                          ...          ...       ...   
335  1936             Ty Cobb\cobbty01    1886-1961     BBWAA   
336  1936     Walter Johnson\johnswa01    1887-1946     BBWAA   
337  1936  Christy Mathewson\mathech01    1880-1925     BBWAA   
338  1936           Babe Ruth\ruthba01    1895-1948     BBWAA   
339  1936       Honus Wagner\wagneho01    1874-1955     BBWAA   

           Inducted As  Votes % of Ballots  
0    Pioneer/Executive    NaN          NaN  
1               Player    NaN          NaN  
2               Player    NaN          NaN  
3               Player    NaN          NaN  
4    Pioneer/Executive    NaN          NaN  
..                 ...    ...          ...  
335             Player  222.0        98.2%  
336             Player  189.0        83.6%  
337             Player  205.0        90.7%  
338             Player  215.0        95.1%  
339             Player  215.0        95.1%  

[340 rows x 7 columns]

In [292]:
HoF_data[['Player','ID']] = HoF_data['Name'].str.split('\\', expand=True)
pd.unique(HoF_data['Inducted As'])

array(['Pioneer/Executive', 'Player', 'Manager', 'Umpire'], dtype=object)

In [293]:
cols = ['Player', 'ID', 'Inducted As']
HoF_ids = HoF_data[cols]
HoF_ids

Player         ID        Inducted As
0           Bud Fowler  fowlebu99  Pioneer/Executive
1           Gil Hodges  hodgegi01             Player
2             Jim Kaat   kaatji01             Player
3        Minnie Minoso  minosmi01             Player
4          Buck O'Neil  oneilbu01  Pioneer/Executive
..                 ...        ...                ...
335            Ty Cobb   cobbty01             Player
336     Walter Johnson  johnswa01             Player
337  Christy Mathewson  mathech01             Player
338          Babe Ruth   ruthba01             Player
339       Honus Wagner  wagneho01             Player

[340 rows x 3 columns]

In [294]:
players = ['Pioneer/Executive', 'Player']
HoF_ids = HoF_ids[HoF_ids['Inducted As'] == 'Player']
HoF_ids = HoF_ids[['ID']]
HoF_ids

ID
1    hodgegi01
2     kaatji01
3    minosmi01
5    olivato01
6    ortizda01
..         ...
335   cobbty01
336  johnswa01
337  mathech01
338   ruthba01
339  wagneho01

[268 rows x 1 columns]

In [297]:
#### Combine with features

# Set this entire list of players as 1 to mark the inductees

HoF_ids['HOF Inductee'] = 1
HoF_ids

ID  HOF Inductee
1    hodgegi01             1
2     kaatji01             1
3    minosmi01             1
5    olivato01             1
6    ortizda01             1
..         ...           ...
335   cobbty01             1
336  johnswa01             1
337  mathech01             1
338   ruthba01             1
339  wagneho01             1

[268 rows x 2 columns]

In [298]:
temp_prepped_stats = pd.merge(HoF_ids, temp_prepped_stats, on = "ID", how = "right")
temp_prepped_stats

ID  HOF Inductee          Player    PA    AB     R     H   2B  \
0        almonbi01           NaN      Bill Almon  3549  3225   366   816  134   
1        armbred01           NaN   Ed Armbrister    91    78     9    20    4   
2         baezjo01           NaN       Jose Baez   391   355    43    87   14   
3        baylodo01           NaN      Don Baylor  6722  5846   904  1506  265   
4        bochtbr01           NaN    Bruce Bochte  4802  4196   525  1198  210   
...            ...           ...             ...   ...   ...   ...   ...  ...   
1655406  youngch04           NaN     Chris Young  3225  2848   417   683  174   
1655407  youngde03           NaN    Delmon Young  2967  2784   342   802  158   
1655408  younger03           NaN  Eric Young Jr.   479   427    67   105   10   
1655409  youngmi02           NaN   Michael Young  7396  6788  1005  2061  388   
1655410  zobribe01           NaN     Ben Zobrist  2458  2108   316   542  120   

         3B   HR  RBI   BB    SO       AVG       SLG       OBP       OPS  
0        25   35  287  246   617  0.253023  0.342636  0.304621  0.647256  
1         3    1    5   10    21  0.256410  0.423077  0.337079  0.760156  
2         2    1   19   25    27  0.245070  0.304225  0.296588  0.600813  
3        13  266  979  583   775  0.257612  0.443893  0.340977  0.784870  
4        16   90  535  526   543  0.285510  0.407531  0.363027  0.770558  
...      ..  ...  ...  ...   ...       ...       ...       ...       ...  
1655406  17  118  367  324   735  0.239817  0.437149  0.319016  0.756165  
1655407   9   71  408  125   514  0.288075  0.427802  0.320971  0.748773  
1655408   4    1   19   47    82  0.245902  0.295082  0.324268  0.619350  
1655409  52  169  917  499  1082  0.303624  0.450796  0.350020  0.800816  
1655410  19   72  314  298   423  0.257116  0.434535  0.347791  0.782326  

[1655411 rows x 17 columns]

In [299]:
temp_prepped_stats = temp_prepped_stats[['ID', 'Player', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'SO', 'AVG', 'SLG', 'OBP', 'OPS', 'HOF Inductee']]
temp_prepped_stats

ID          Player    PA    AB     R     H   2B  3B   HR  RBI  \
0        almonbi01      Bill Almon  3549  3225   366   816  134  25   35  287   
1        armbred01   Ed Armbrister    91    78     9    20    4   3    1    5   
2         baezjo01       Jose Baez   391   355    43    87   14   2    1   19   
3        baylodo01      Don Baylor  6722  5846   904  1506  265  13  266  979   
4        bochtbr01    Bruce Bochte  4802  4196   525  1198  210  16   90  535   
...            ...             ...   ...   ...   ...   ...  ...  ..  ...  ...   
1655406  youngch04     Chris Young  3225  2848   417   683  174  17  118  367   
1655407  youngde03    Delmon Young  2967  2784   342   802  158   9   71  408   
1655408  younger03  Eric Young Jr.   479   427    67   105   10   4    1   19   
1655409  youngmi02   Michael Young  7396  6788  1005  2061  388  52  169  917   
1655410  zobribe01     Ben Zobrist  2458  2108   316   542  120  19   72  314   

          BB    SO       AVG       SLG       OBP       OPS  HOF Inductee  
0        246   617  0.253023  0.342636  0.304621  0.647256           NaN  
1         10    21  0.256410  0.423077  0.337079  0.760156           NaN  
2         25    27  0.245070  0.304225  0.296588  0.600813           NaN  
3        583   775  0.257612  0.443893  0.340977  0.784870           NaN  
4        526   543  0.285510  0.407531  0.363027  0.770558           NaN  
...      ...   ...       ...       ...       ...       ...           ...  
1655406  324   735  0.239817  0.437149  0.319016  0.756165           NaN  
1655407  125   514  0.288075  0.427802  0.320971  0.748773           NaN  
1655408   47    82  0.245902  0.295082  0.324268  0.619350           NaN  
1655409  499  1082  0.303624  0.450796  0.350020  0.800816           NaN  
1655410  298   423  0.257116  0.434535  0.347791  0.782326           NaN  

[1655411 rows x 17 columns]

In [300]:
pd.unique(temp_prepped_stats['HOF Inductee'])

array([nan,  1.])

In [301]:
hof_is_nan = temp_prepped_stats.loc[pd.isna(temp_prepped_stats['HOF Inductee'])]
series_of_hof = hof_is_nan.index
series_of_hof



Int64Index([      0,       1,       2,       3,       4,       5,       6,
                  7,       8,       9,
            ...
            1655401, 1655402, 1655403, 1655404, 1655405, 1655406, 1655407,
            1655408, 1655409, 1655410],
           dtype='int64', length=1559958)

In [302]:
temp_prepped_stats.loc[series_of_hof, 'HOF Inductee'] = 0
temp_prepped_stats['HOF Inductee'] = temp_prepped_stats['HOF Inductee'].astype(int)
temp_prepped_stats

ID          Player    PA    AB     R     H   2B  3B   HR  RBI  \
0        almonbi01      Bill Almon  3549  3225   366   816  134  25   35  287   
1        armbred01   Ed Armbrister    91    78     9    20    4   3    1    5   
2         baezjo01       Jose Baez   391   355    43    87   14   2    1   19   
3        baylodo01      Don Baylor  6722  5846   904  1506  265  13  266  979   
4        bochtbr01    Bruce Bochte  4802  4196   525  1198  210  16   90  535   
...            ...             ...   ...   ...   ...   ...  ...  ..  ...  ...   
1655406  youngch04     Chris Young  3225  2848   417   683  174  17  118  367   
1655407  youngde03    Delmon Young  2967  2784   342   802  158   9   71  408   
1655408  younger03  Eric Young Jr.   479   427    67   105   10   4    1   19   
1655409  youngmi02   Michael Young  7396  6788  1005  2061  388  52  169  917   
1655410  zobribe01     Ben Zobrist  2458  2108   316   542  120  19   72  314   

          BB    SO       AVG       SLG       OBP       OPS  HOF Inductee  
0        246   617  0.253023  0.342636  0.304621  0.647256             0  
1         10    21  0.256410  0.423077  0.337079  0.760156             0  
2         25    27  0.245070  0.304225  0.296588  0.600813             0  
3        583   775  0.257612  0.443893  0.340977  0.784870             0  
4        526   543  0.285510  0.407531  0.363027  0.770558             0  
...      ...   ...       ...       ...       ...       ...           ...  
1655406  324   735  0.239817  0.437149  0.319016  0.756165             0  
1655407  125   514  0.288075  0.427802  0.320971  0.748773             0  
1655408   47    82  0.245902  0.295082  0.324268  0.619350             0  
1655409  499  1082  0.303624  0.450796  0.350020  0.800816             0  
1655410  298   423  0.257116  0.434535  0.347791  0.782326             0  

[1655411 rows x 17 columns]

In [303]:
pd.unique(temp_prepped_stats['HOF Inductee'])

array([0, 1])

In [90]:
## ALL stats (in df_mid...)
summables = ['PA','AB','R','H','2B','3B','HR','RBI','BB','SO','HBP','SH', 'SF']
temp_career_stats = temp[summables].sum()
temp_career_stats



PA   AB   R    H  2B  3B  HR  RBI  BB   SO  HBP  SH  SF
ID                                                                 
aardsda01    4    3   0    0   0   0   0    0   0    1    0   1   0
aasedo01     5    5   0    0   0   0   0    0   0    3    0   0   0
abadan01    25   21   1    2   0   0   0    0   4    5    0   0   0
abadfe01     1    1   0    0   0   0   0    0   0    1    0   0   0
abbotje01  651  596  82  157  33   2  18   83  38   91    3   5   7
...        ...  ...  ..  ...  ..  ..  ..  ...  ..  ...  ...  ..  ..
zoskyed01   53   50   4    8   1   2   0    3   1   13    0   1   1
zuberjo01  151  136  13   34   7   1   3   16  12   20    1   1   1
zuletju01  191  174  23   43  11   0   9   36  10   51    6   0   1
zupcibo01  886  795  93  199  47   4   7   80  57  137    6  20   8
zuvelpa01  545  491  40  109  17   2   2   20  34   50    2  18   0

[5825 rows x 13 columns]

### Different Statistical Formulae

singles (int) = H - (2B + 3B + HR)

total_bases (int) = 1 * singles + 2 * 2B + 3 * 3B + 4 * HR


**AVG (float) = H / AB**

SLG (float) = total_bases / AB

OBP (float) = (H + BB + HBP)/(AB + BB + HBP + SF)

**OPS (float) = SLG + OBP**
https://en.wikipedia.org/wiki/On-base_plus_slugging  *** Reference is great for testing against

RC (float) = total_bases * ( (H + BB) / (AB + BB) )

ISO (float) = (1 * 2B + 2 * 3B + 3 * HR) / AB

PA/SO (float) = PA/SO

#### Simplification of SLG formula??

SLG = ( 1 * (H - (2B + 3B + HR)) + 2 * 2B + 3 * 3B + 4 * HR ) / AB
( H - 2B - 3B - HR + 2 * 2B + 3 * 3B + 4 * HR ) / AB
( H - 2B + 2 * 2B - 3B + 3 * 3B - HR + 4 * HR ) / AB
( H + 2B + 2 * 3B + 3 * HR ) / AB

In [124]:
## add lifetime percentages

temp_career_stats['AVG'] = temp_career_stats['H'] / (temp_career_stats['AB']*1.0)

temp_career_stats['SLG'] = (temp_career_stats['H'] + temp_career_stats['2B'] + 2*temp_career_stats['3B'] + 3*temp_career_stats['HR']) / (temp_career_stats['AB']*1.0)

temp_career_stats['OBP'] = (temp_career_stats['H'] + temp_career_stats['BB'] + temp_career_stats['HBP']) / ((temp_career_stats['AB'] + temp_career_stats['BB'] + temp_career_stats['HBP'] + temp_career_stats['SF'])*1.0) 

temp_career_stats['OPS'] = temp_career_stats['SLG'] + temp_career_stats['OBP']

temp_career_stats['RC'] = (temp_career_stats['H'] + temp_career_stats['2B'] + 2*temp_career_stats['3B'] + 3*temp_career_stats['HR']) * ( (temp_career_stats['H'] + temp_career_stats['BB']) / (temp_career_stats['AB'] + temp_career_stats['BB']) )

temp_career_stats['ISO'] = (temp_career_stats['2B'] + 2*temp_career_stats['3B'] + 3*temp_career_stats['HR']) / (temp_career_stats['AB']*1.0)

temp_career_stats['PA_SO'] = temp_career_stats['PA'] / temp_career_stats['SO']*1.0
temp_career_stats['PA_BB'] = temp_career_stats['PA'] / temp_career_stats['BB']*1.0
temp_career_stats['SO_BB'] = temp_career_stats['SO'] / temp_career_stats['BB']*1.0


temp_career_stats['PA_RBI'] = temp_career_stats['PA'] / temp_career_stats['RBI']*1.0


temp_career_stats

PA   AB   R    H  2B  3B  HR  RBI  BB   SO  HBP  SH  SF       AVG  \
ID                                                                              
aardsda01    4    3   0    0   0   0   0    0   0    1    0   1   0  0.000000   
aasedo01     5    5   0    0   0   0   0    0   0    3    0   0   0  0.000000   
abadan01    25   21   1    2   0   0   0    0   4    5    0   0   0  0.095238   
abadfe01     1    1   0    0   0   0   0    0   0    1    0   0   0  0.000000   
abbotje01  651  596  82  157  33   2  18   83  38   91    3   5   7  0.263423   
...        ...  ...  ..  ...  ..  ..  ..  ...  ..  ...  ...  ..  ..       ...   
zoskyed01   53   50   4    8   1   2   0    3   1   13    0   1   1  0.160000   
zuberjo01  151  136  13   34   7   1   3   16  12   20    1   1   1  0.250000   
zuletju01  191  174  23   43  11   0   9   36  10   51    6   0   1  0.247126   
zupcibo01  886  795  93  199  47   4   7   80  57  137    6  20   8  0.250314   
zuvelpa01  545  491  40  109  17   2   2   20  34   50    2  18   0  0.221996   

                SLG       OBP       OPS         RC       ISO      PA_SO  \
ID                                                                        
aardsda01  0.000000  0.000000  0.000000   0.000000  0.000000   4.000000   
aasedo01   0.000000  0.000000  0.000000   0.000000  0.000000   1.666667   
abadan01   0.095238  0.240000  0.335238   0.480000  0.000000   5.000000   
abadfe01   0.000000  0.000000  0.000000   0.000000  0.000000   1.000000   
abbotje01  0.416107  0.307453  0.723561  76.277603  0.152685   7.153846   
...             ...       ...       ...        ...       ...        ...   
zoskyed01  0.260000  0.173077  0.433077   2.294118  0.100000   4.076923   
zuberjo01  0.382353  0.313333  0.695686  16.162162  0.132353   7.550000   
zuletju01  0.465517  0.308901  0.774418  23.331522  0.218391   3.745098   
zupcibo01  0.345912  0.302540  0.648452  82.629108  0.095597   6.467153   
zuvelpa01  0.276986  0.275142  0.552128  37.043810  0.054990  10.900000   

               PA_BB      SO_BB    RBI_PA     PA_RBI  
ID                                                    
aardsda01        inf        inf  0.000000        inf  
aasedo01         inf        inf  0.000000        inf  
abadan01    6.250000   1.250000  0.000000        inf  
abadfe01         inf        inf  0.000000        inf  
abbotje01  17.131579   2.394737  0.127496   7.843373  
...              ...        ...       ...        ...  
zoskyed01  53.000000  13.000000  0.056604  17.666667  
zuberjo01  12.583333   1.666667  0.105960   9.437500  
zuletju01  19.100000   5.100000  0.188482   5.305556  
zupcibo01  15.543860   2.403509  0.090293  11.075000  
zuvelpa01  16.029412   1.470588  0.036697  27.250000  

[5825 rows x 24 columns]

In [175]:
df[df['Player'] == "Miguel Cabrera"]

ID          Player   Tm  Opp  PA  AB  R  H  2B  3B  HR  RBI  \
3365667  cabremi01  Miguel Cabrera  FLA  TBD   5   5  1  1   0   0   1    2   
3365991  cabremi01  Miguel Cabrera  FLA  TBD   2   2  0  0   0   0   0    0   
3366302  cabremi01  Miguel Cabrera  FLA  TBD   3   3  1  1   0   1   0    1   
3366788  cabremi01  Miguel Cabrera  FLA  NYM   5   5  1  2   1   0   0    0   
3367108  cabremi01  Miguel Cabrera  FLA  NYM   4   4  0  1   0   0   0    0   
...            ...             ...  ...  ...  ..  .. .. ..  ..  ..  ..  ...   
4283662  cabremi01  Miguel Cabrera  DET  CHW   4   4  0  2   1   0   0    0   
4283786  cabremi01  Miguel Cabrera  DET  MIN   4   4  0  1   0   0   0    0   
4284125  cabremi01  Miguel Cabrera  DET  MIN   5   4  0  0   0   0   0    0   
4284700  cabremi01  Miguel Cabrera  DET  CHW   4   4  0  0   0   0   0    0   
4285016  cabremi01  Miguel Cabrera  DET  CHW   4   4  0  0   0   0   0    0   

         BB  SO  HBP  SH  SF Result  Season  
3365667   0   1    0   0   0      W    2003  
3365991   0   1    0   0   0      W    2003  
3366302   0   1    0   0   0      W    2003  
3366788   0   1    0   0   0      W    2003  
3367108   0   0    0   0   0      L    2003  
...      ..  ..  ...  ..  ..    ...     ...  
4283662   0   0    0   0   0      L    2021  
4283786   0   0    0   0   0      L    2021  
4284125   0   1    1   0   0      L    2021  
4284700   0   4    0   0   0      L    2021  
4285016   0   0    0   0   0      L    2021  

[2584 rows x 19 columns]

In [137]:

temp2 = df_mid.groupby(['ID', 'Player', 'Season'])
temp2

In [138]:
t2 = temp2.sum()
t2

PA   AB   R   H  2B  3B  HR  RBI  BB  SO  \
ID        Player        Season                                              
aardsda01 David Aardsma 2006      3    2   0   0   0   0   0    0   0   0   
                        2008      1    1   0   0   0   0   0    0   0   1   
aasedo01  Don Aase      1989      5    5   0   0   0   0   0    0   0   3   
abadan01  Andy Abad     2001      1    1   0   0   0   0   0    0   0   0   
                        2003     19   17   1   2   0   0   0    0   2   5   
...                             ...  ...  ..  ..  ..  ..  ..  ...  ..  ..   
zuvelpa01 Paul Zuvella  1985    210  190  16  48   8   1   0    4  16  14   
                        1986     57   48   2   4   1   0   0    2   5   4   
                        1987     36   34   2   6   0   0   0    0   0   4   
                        1988    146  130   9  30   5   1   0    7   8  13   
                        1989     60   58   9  16   2   0   2    6   1  11   

                                HBP  SH  SF  
ID        Player        Season               
aardsda01 David Aardsma 2006      0   1   0  
                        2008      0   0   0  
aasedo01  Don Aase      1989      0   0   0  
abadan01  Andy Abad     2001      0   0   0  
                        2003      0   0   0  
...                             ...  ..  ..  
zuvelpa01 Paul Zuvella  1985      0   4   0  
                        1986      0   4   0  
                        1987      0   2   0  
                        1988      0   8   0  
                        1989      1   0   0  

[28999 rows x 13 columns]

In [139]:
## add lifetime percentages

t2['AVG'] = t2['H'] / (t2['AB']*1.0)

t2['SLG'] = (t2['H'] + t2['2B'] + 2*t2['3B'] + 3*t2['HR']) / (t2['AB']*1.0)

t2['OBP'] = (t2['H'] + t2['BB'] + t2['HBP']) / ((t2['AB'] + t2['BB'] + t2['HBP'] + t2['SF'])*1.0) 

t2['OPS'] = t2['SLG'] + t2['OBP']

t2['RC'] = (t2['H'] + t2['2B'] + 2*t2['3B'] + 3*t2['HR']) * ( (t2['H'] + t2['BB']) / (t2['AB'] + t2['BB']) )

t2['ISO'] = (t2['2B'] + 2*t2['3B'] + 3*t2['HR']) / (t2['AB']*1.0)

t2['PA_SO'] = t2['PA'] / t2['SO']*1.0
t2['PA_BB'] = t2['PA'] / t2['BB']*1.0
t2['SO_BB'] = t2['SO'] / t2['BB']*1.0


t2['PA_RBI'] = t2['PA'] / t2['RBI']*1.0


t2

PA   AB   R   H  2B  3B  HR  RBI  BB  SO  \
ID        Player        Season                                              
aardsda01 David Aardsma 2006      3    2   0   0   0   0   0    0   0   0   
                        2008      1    1   0   0   0   0   0    0   0   1   
aasedo01  Don Aase      1989      5    5   0   0   0   0   0    0   0   3   
abadan01  Andy Abad     2001      1    1   0   0   0   0   0    0   0   0   
                        2003     19   17   1   2   0   0   0    0   2   5   
...                             ...  ...  ..  ..  ..  ..  ..  ...  ..  ..   
zuvelpa01 Paul Zuvella  1985    210  190  16  48   8   1   0    4  16  14   
                        1986     57   48   2   4   1   0   0    2   5   4   
                        1987     36   34   2   6   0   0   0    0   0   4   
                        1988    146  130   9  30   5   1   0    7   8  13   
                        1989     60   58   9  16   2   0   2    6   1  11   

                                HBP  SH  SF       AVG       SLG       OBP  \
ID        Player        Season                                              
aardsda01 David Aardsma 2006      0   1   0  0.000000  0.000000  0.000000   
                        2008      0   0   0  0.000000  0.000000  0.000000   
aasedo01  Don Aase      1989      0   0   0  0.000000  0.000000  0.000000   
abadan01  Andy Abad     2001      0   0   0  0.000000  0.000000  0.000000   
                        2003      0   0   0  0.117647  0.117647  0.210526   
...                             ...  ..  ..       ...       ...       ...   
zuvelpa01 Paul Zuvella  1985      0   4   0  0.252632  0.305263  0.310680   
                        1986      0   4   0  0.083333  0.104167  0.169811   
                        1987      0   2   0  0.176471  0.176471  0.176471   
                        1988      0   8   0  0.230769  0.284615  0.275362   
                        1989      1   0   0  0.275862  0.413793  0.300000   

                                     OPS         RC       ISO      PA_SO  \
ID        Player        Season                                             
aardsda01 David Aardsma 2006    0.000000   0.000000  0.000000        inf   
                        2008    0.000000   0.000000  0.000000   1.000000   
aasedo01  Don Aase      1989    0.000000   0.000000  0.000000   1.666667   
abadan01  Andy Abad     2001    0.000000   0.000000  0.000000        inf   
                        2003    0.328173   0.421053  0.000000   3.800000   
...                                  ...        ...       ...        ...   
zuvelpa01 Paul Zuvella  1985    0.615943  18.019417  0.052632  15.000000   
                        1986    0.273978   0.849057  0.020833  14.250000   
                        1987    0.352941   1.058824  0.000000   9.000000   
                        1988    0.559978  10.188406  0.053846  11.230769   
                        1989    0.713793   6.915254  0.137931   5.454545   

                                 PA_BB   SO_BB     PA_RBI  
ID        Player        Season                             
aardsda01 David Aardsma 2006       inf     NaN        inf  
                        2008       inf     inf        inf  
aasedo01  Don Aase      1989       inf     inf        inf  
abadan01  Andy Abad     2001       inf     NaN        inf  
                        2003     9.500   2.500        inf  
...                                ...     ...        ...  
zuvelpa01 Paul Zuvella  1985    13.125   0.875  52.500000  
                        1986    11.400   0.800  28.500000  
                        1987       inf     inf        inf  
                        1988    18.250   1.625  20.857143  
                        1989    60.000  11.000  10.000000  

[28999 rows x 23 columns]

### Visualization(s)

??? Before and after bar graphs?

## Model Selection

### Visualization(s)

## Model Evaluation

### Visualization(s)

## Concluding Comments